<a href="https://colab.research.google.com/github/fabriziobasso/Closet_Index_Tracking/blob/master/File_00_NN_ext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<h1 align="center"><font color='#001ddd'> GLUCOSE PREDICTION DATASET**</font></h1>

## **Dataset Description**
The dataset is from a study that collected data from young adults in the UK with type 1 diabetes, who used a continuous glucose monitor (CGM), an insulin pump and a smartwatch. These devices collected blood glucose readings, insulin dosage, carbohydrate intake, and activity data. The data collected was aggregated to five-minute intervals and formatted into samples. Each sample represents a point in time and includes the aggregated five-minute intervals from the previous six hours. The aim is to predict the blood glucose reading an hour into the future, for each of these samples.

The training set takes samples from the first three months of study data from nine of the participants and includes the future blood glucose value. These training samples appear in chronological order and overlap. The testing set takes samples from the remainder of the study period from fifteen of the participants (so unseen participants appear in the testing set). These testing samples do not overlap and are in a random order to avoid data leakage.

**Complexities to be aware of:**

This is medical data so there are missing values and noise in the data
the participants did not all use the same device models (CGM, insulin pump and smartwatch) so there may be differences in the collection method of the data
some participants in the test set do not appear in the training set

In [ ]:
%%capture
# Connect to Colab:#
from google.colab import drive
import os
drive.mount('/content/drive')

!pip install category-encoders
!pip install optuna
!pip install optuna-integration
#!pip install scikit-learn==1.4
!pip install catboost
!pip install deeptables

!pip install keras-tuner --upgrade
!pip install keras-nlp
!pip install BorutaShap
!pip install scikit-lego
!!pip install --no-index -U --find-links=/kaggle/input/deeptables-v0-2-5/deeptables-0.2.5 deeptables==0.2.5

In [ ]:
folder_script = models_folders = "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose"
os.chdir(folder_script)

In [ ]:
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.wrapper import PolynomialWrapper
from category_encoders.count import CountEncoder

# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
#import calplot as cal
import missingno as msno
import category_encoders as ce

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)


# Bloomberg
#from xbbg import blp
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping

import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
import datetime
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError, MeanAbsoluteError

from keras.utils import FeatureSpace, plot_model

# Import libraries for Hypertuning
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, GridSearch, BayesianOptimization

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   LabelEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)

# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  QuantileRegressor,
                                  ARDRegression,
                                  TheilSenRegressor,
                                  PoissonRegressor,
                                  GammaRegressor)

from sklearn.ensemble import (AdaBoostRegressor,
                              AdaBoostClassifier,
                              RandomForestRegressor,
                              RandomForestClassifier,
                              VotingRegressor,
                              GradientBoostingRegressor,
                              GradientBoostingClassifier,
                              StackingRegressor,
                              HistGradientBoostingClassifier,
                              HistGradientBoostingRegressor,
                              ExtraTreesClassifier)

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
import logging
from openpyxl import load_workbook

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

warnings.filterwarnings('ignore')
#plt.style.use('fivethirtyeight')

# Setting rc parameters in seaborn for plots and graphs-
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

In [ ]:
tf.__version__

In [ ]:
!apt install graphviz
import os
os.environ["PATH"] += os.pathsep + '/usr/bin/dot'

# 1.0 Upload Data

## 1.1 Functions:

In [ ]:
def encode_target(y_train, y_test, encoder_type='label', enc_strategy=False):
    """
    Encodes the target columns in the training and testing data
    using the specified encoder type.

    Parameters:
    y_train (pd.Series or pd.DataFrame): Training target data.
    y_test (pd.Series or pd.DataFrame): Testing target data.

    Returns:
    y_train_encoded (pd.Series): Encoded training target data.
    y_test_encoded (pd.Series): Encoded testing target data.
    """

    if encoder_type == 'label':
        encoder = LabelEncoder()
        y_train_encoded = encoder.fit_transform(y_train)
        y_test_encoded = encoder.transform(y_test)

        y_train_encoded = pd.Series(y_train_encoded, index=y_train.index, name="Target")
        y_test_encoded = pd.Series(y_test_encoded, index=y_test.index, name="Target")


    elif encoder_type == 'onehot':
        y_train_ = y_train.values.reshape(-1, 1)
        y_test_ = y_test.values.reshape(-1, 1)

        encoder = OneHotEncoder(sparse_output=False)
        y_train_encoded = encoder.fit_transform(y_train_)
        y_test_encoded = encoder.transform(y_test_)

        y_train_encoded = pd.DataFrame(y_train_encoded, index=y_train.index)
        y_test_encoded = pd.DataFrame(y_test_encoded, index=y_test.index)

    else:
        raise ValueError("Invalid encoder_type. Currently supported: 'label'.")

    if enc_strategy:
        return y_train_encoded, y_test_encoded, encoder

    else:
        return y_train_encoded, y_test_encoded

def encode_data(X_train, X_test, encoder_type='label', columns=None, map=None):
    """
    Encodes the training and testing data using the specified encoder type.

    Parameters:
    X_train (pd.DataFrame): Training data.
    X_test (pd.DataFrame): Testing data.
    encoder_type (str): Type of encoder ('label' or 'onehot'). Default is 'label'.
    columns (list): List of columns to encode. If None, all object type columns are encoded.

    Returns:
    X_train_encoded (pd.DataFrame): Encoded training data.
    X_test_encoded (pd.DataFrame): Encoded testing data.
    """

    if columns is None:
        # Default to all object type columns if no columns are specified
        columns = X_train.select_dtypes(include=['object']).columns.tolist()

    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()

    if encoder_type == 'label':
        for col in columns:
            le = LabelEncoder()
            X_train_encoded[col] = le.fit_transform(X_train[col])
            X_test_encoded[col] = le.transform(X_test[col])

    elif encoder_type == 'onehot':
        for col in columns:
            ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
            # Fit the encoder on the training data and transform both training and test data
            encoded_train = ohe.fit_transform(X_train[[col]])
            encoded_test = ohe.transform(X_test[[col]])

            # Create a DataFrame with the encoded data
            encoded_train_df = pd.DataFrame(encoded_train, columns=ohe.get_feature_names_out([col]))
            encoded_test_df = pd.DataFrame(encoded_test, columns=ohe.get_feature_names_out([col]))

            # Concatenate the new columns to the original dataframes and drop the original columns
            X_train_encoded = pd.concat([X_train_encoded.drop(col, axis=1), encoded_train_df], axis=1)
            X_test_encoded = pd.concat([X_test_encoded.drop(col, axis=1), encoded_test_df], axis=1)

    elif encoder_type == 'count_encoder':

          for col in columns:

                target_encoder = CountEncoder(cols=columns)
                X_train_encoded = target_encoder.fit_transform(X_train_encoded)
                X_test_encoded = target_encoder.transform(X_test_encoded)

    else:
        raise ValueError("Invalid encoder_type. Currently supported: 'label', 'onehot', 'target_encoder'.")

    return X_train_encoded, X_test_encoded

def plot_training_session(history):
  # Plot training and validation loss scores
  # against the number of epochs.
  plt.figure(figsize=(8, 6))
  plt.plot(history.history['loss'], label='Train')
  plt.plot(history.history['val_loss'], label='Validation')
  plt.grid(linestyle='--')
  plt.ylabel('val_loss')
  plt.xlabel('Epoch')
  plt.title('Train-Validation Scores', pad=13)
  plt.legend(loc='upper right');
  plt.show()

## **1.2 Importing the Dataset**

### **1.2.1 Files**
* activities.txt - a list of activity names that appear in the activity-X:XX columns
* sample_submission.csv - a sample submission file in the correct format
* test.csv - the test set
* train.csv - the training set

## **Columns**
* train.csv:
    * **id - row id** consisting of participant number and a count for that participant
    * **p_num** - participant number
    * **time** - time of day in the format HH:MM:SS
    * **bg-X:XX** - blood glucose reading in mmol/L, X:XX(H:SS) time in the past (e.g. bg-2:35, would be the blood glucose reading from 2 hours and 35 minutes before the time value for that row), recorded by the continuous glucose monitor
    * **insulin-X:XX** - total insulin dose received in units in the last 5 minutes, X:XX(H:SS) time in the past (e.g. insulin-2:35, would be the total insulin dose received between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the insulin pump
    * **carbs-X:XX** - total carbohydrate value consumed in grammes in the last 5 minutes, X:XX(H:SS) time in the past (e.g. carbs-2:35, would be the total carbohydrate value consumed between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the participant
    * **hr-X:XX** - mean heart rate in beats per minute in the last 5 minutes, X:XX(H:SS) time in the past (e.g. hr-2:35, would be the mean heart rate between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the smartwatch
    * **steps-X:XX** - total steps walked in the last 5 minutes, X:XX(H:SS) time in the past (e.g. * steps-2:35, would be the total steps walked between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the smartwatch
    * **cals-X:XX** - total calories burnt in the last 5 minutes, X:XX(H:SS) time in the past (e.g. cals-2:35, would be the total calories burned between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), calculated by the smartwatch
    * **activity-X:XX** - self-declared activity performed in the last 5 minutes, X:XX(H:SS) time in the past (e.g. activity-2:35, would show a string name of the activity performed between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), set on the smartwatch
    * **bg+1:00** - blood glucose reading in mmol/L an hour in the future, this is the value you will be predicting (not provided in test.csv)

In [ ]:
ext_data=True

if ext_data==False:
  df_train=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train_cluster.csv", index_col=0)
  df_test=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test_cluster.csv", index_col=0)

  df_train_scaled=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled.csv", index_col=0)
  df_test.shape

if ext_data==True:
  df_train = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train_cluster_enc_final_ext.csv", index_col=0)
  df_test = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test_cluster_enc_final_ext.csv", index_col=0)

#  df_train_scaled=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled.csv", index_col=0)
  print(df_test.shape,df_train.shape)

In [ ]:
df_train.p_num.value_counts()/df_train.shape[0]

In [ ]:
df_train.p_num = df_train.p_num.astype("str")
df_test.p_num = df_test.p_num.astype("str")

In [ ]:
#df_train.groupby(["cluster"])["p_num"].count()

In [ ]:
#df_train.groupby(["cluster","p_num"])["p_num"].count()[8]

In [ ]:
df_train.head()
df_train.columns

#### **SCALE FEATURES**

In [ ]:
float_feat = ['PC_1', 'PC_2', 'PC_3', 'PC_4', 'enc_08_v6', 'enc_07_v7', 'enc_03_v7', 'enc_01_v5',
              'enc_02_v5', 'enc_03_v5', 'enc_04_v5', 'enc_05_v5', 'enc_06_v5', 'enc_07_v5', 'enc_08_v5']

scaler_float = StandardScaler()

df_train[float_feat] = scaler_float.fit_transform(df_train[float_feat])
df_test[float_feat] = scaler_float.transform(df_test[float_feat])

Sub-dataset are created for each main set of features to inpute missing values:

In [ ]:
print(df_train.shape)
display(df_train.groupby("p_num")["p_num"].count())
#display(dict(df_test.groupby("p_num")["p_num"].count()))
#display(df_train[df_train["p_num"]=="p01"].head())
#display(df_train[df_train["p_num"]=="p02"].head())

vocabulary_pnum = list(dict(df_train.groupby("p_num")["p_num"].count()).keys())
len(vocabulary_pnum)

##### Features:

In [ ]:
(df_train.columns==df_test.columns).all()

##### Define Features Datatypes:

In [ ]:
ts_fields = list(df_test.columns)
df_train.info()

In [ ]:
X_train = df_train.copy()
X_test = df_test.copy()

In [ ]:
int_cols = X_train.select_dtypes(include=['int']).columns.tolist()
float_cols = X_train.select_dtypes(include=['float']).columns.tolist()
obj_cols = X_train.select_dtypes(include=['object']).columns.tolist()

X_train[int_cols] = X_train[int_cols].astype("int32")
X_train[float_cols] = X_train[float_cols].astype("float32")
X_train[obj_cols] = X_train[obj_cols].astype("string")

X_test[int_cols] = X_test[int_cols].astype("int32")
X_test[float_cols] = X_test[float_cols].astype("float32")
X_test[obj_cols] = X_test[obj_cols].astype("string")

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.info()

# **2.0 Neural Network Preparation:**

###### **FUNCTIONS**

In [ ]:
def residual_block(input_tensor, filters, kernel_size, strides=1):
    x = layers.Conv1D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = layers.LayerNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SpatialDropout1D(0.2)(x)

    # Residual connection
    if strides > 1 or input_tensor.shape[-1] != filters:
        residual = layers.Conv1D(filters, 1, strides=strides, padding='same')(input_tensor)
    else:
        residual = input_tensor

    x = layers.add([x, residual])
    x = layers.MaxPooling1D(pool_size=2)(x)
    return x

def dense_block(cells, dropout=0.3,activation="selu", reg=0.0, name=None, normalization_type='batch'):
  if activation == "selu":
    block = keras.Sequential(name=name)
    block.add(Dense(cells,
                    activity_regularizer=keras.regularizers.l2(reg),
                    name=f"{name}_dense",
                    kernel_initializer="lecun_normal"))

    if normalization_type == 'batch':
      block.add(layers.BatchNormalization(name=f"{name}_bn"))
    elif normalization_type == 'layer':
      block.add(layers.LayerNormalization(name=f"{name}_ln"))
    else:
      raise ValueError("Invalid normalization_type. Choose 'batch' or 'layer'.")

    block.add(layers.Activation(activation, name=f"{name}_activ"))
    block.add(layers.AlphaDropout(dropout, name=f"{name}_do_alpha"))

  else:
    block = keras.Sequential(name=name)
    block.add(Dense(cells,
                    activity_regularizer=keras.regularizers.l2(reg),
                    name=f"{name}_dense",
                    kernel_initializer="he_normal"))

    if normalization_type == 'batch':
      block.add(layers.BatchNormalization(name=f"{name}_bn"))
    elif normalization_type == 'layer':
      block.add(layers.LayerNormalization(name=f"{name}_ln"))
    else:
      raise ValueError("Invalid normalization_type. Choose 'batch' or 'layer'.")

    block.add(layers.Activation(activation, name=f"{name}_activ"))
    block.add(layers.Dropout(dropout, name=f"{name}_do"))

  return block

def residual_block_dense(input_tensor, units, normalization_type='batch', dropout_rate=0.2,activation='relu', name="block",reg=0.0):
    """
    Creates a residual block with dense layers.

    Args:
      input_tensor: Input tensor to the block.
      units: Number of units in the dense layer.
      normalization_type: Type of normalization to use ('batch' or 'layer').
      dropout_rate: Dropout rate for the dropout layer.

    Returns:
      Output tensor of the block.
    """

    x = layers.Dense(units, activation=activation,name=f"dense_01_{name}",activity_regularizer=keras.regularizers.l2(reg))(input_tensor)
    if normalization_type == 'batch':
        x = layers.BatchNormalization(name=f"bn_01_{name}")(x)
    elif normalization_type == 'layer':
        x = layers.LayerNormalization(name=f"ln_01_{name}")(x)
    else:
        raise ValueError("Invalid normalization_type. Choose 'batch' or 'layer'.")
    x = layers.Dropout(dropout_rate,name=f"do_01_{name}")(x)

    # Residual connection
    if input_tensor.shape[-1] != units:
        residual = layers.Dense(units,name=f"dense_02_{name}")(input_tensor)
    else:
        residual = input_tensor

    x = layers.add([x, residual],name=f"add_01_{name}")
    return x

## **2.1 MODEL 01**

### 2.1.1 Create Dataloader:

In [ ]:
gc.collect()

In [ ]:
def dataframe_to_dataset(dataframe_, shuffle=False, batch_size=32):
    dataframe = dataframe_.copy()
    target = dataframe["bg+1:00"]
    dataframe = dataframe.drop(columns=["bg+1:00"])
    tab_data = dataframe.drop(columns=["p_num"])

    ds = tf.data.Dataset.from_tensor_slices(((dataframe["p_num"].values,  # First input
                                              tab_data.values), # Second input
                                              target))

    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [ ]:
train_ds = dataframe_to_dataset(X_train,batch_size=1)

* **TEST THE DATALOADER:**

In [ ]:
for (x0, x1), y in train_ds.take(1):
    print(x0.shape)
    print(x1.shape)
    print(y.shape)

In [ ]:
X_train.info()

### **2.1.3 Create Models:**

In [ ]:
def create_model_v0(vocabulary=vocabulary_pnum,
                    dense_layers=[256,128,128,64],
                    dropout=0.3,
                    activation="selu",
                    reg=0.0,
                    lr=0.001,
                    noise=0.05,
                    summary=False):

  # --- Tabular Data Processing ---
  # Input layer for tabular data as a dictionary

  pnum_input = keras.Input(shape=(1,), name="pnum_input", dtype="string")

  # Lookup Layer for the p_num:
  lookup = layers.StringLookup(vocabulary=vocabulary,mask_token=None,num_oov_indices=1,
                output_mode="int", name="lookup_pnum")
  pnum_encoded = lookup(pnum_input)

  # Embedding layers for hour and minute
  pnum_encoded_embedding = layers.Embedding(input_dim=len(vocabulary)+1,
                                            output_dim=5,
                                            name="embed_pnum")(pnum_encoded)
  # Flatten the embeddings
  pnum_flat = layers.Flatten()(pnum_encoded_embedding)

  # Input layer for Tabular Data
  tab_data = keras.Input(shape=(15,), name="tab_input")

  # Concatenate all tabular features
  x = layers.concatenate([tab_data, pnum_flat])
  x = layers.BatchNormalization()(x)

  x = layers.GaussianNoise(stddev=noise, seed=42, name="gn_00")(x)
  tabular_output = x
  tabular_output = layers.Dropout(dropout, name="bb_00")(tabular_output)

  # Dense layers for tabular data
  for num, den in enumerate(dense_layers):
    tabular_output = dense_block(den,dropout=dropout,activation=activation, reg=reg, name=f"block_{num}")(tabular_output)


  tabular_output = layers.concatenate([x, tabular_output])
  # --- Output Layer ---
  # Final dense layer for prediction
  output = layers.Dense(1, name="output")(tabular_output)

  # --- Create and Compile the Model ---
  # Create the model
  model = keras.Model(inputs=[pnum_input, tab_data], outputs=output)
  optimizer= keras.optimizers.Adam(learning_rate=lr)
  metric = RootMeanSquaredError(name="rmse", dtype=None)
  metric_mae = MeanAbsoluteError(name="mae", dtype=None)

  # Compile the model
  model.compile(optimizer=optimizer, loss="mse", metrics=[metric,metric_mae])

  # Print model summary
  if summary==True:
    model.summary()

  return model

model = create_model_v0(vocabulary=vocabulary_pnum,summary=True)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True, rankdir="LR", expand_nested=True, show_layer_activations=True)

### 2.1.4 Training Functions:

#### Main Function:

In [ ]:
def run_experiment(X_train, X_test, model_constructor, best_params, experiment_name="baseline_nn_ext", rs=42, target="bg+1:00",
                   batch_size=64, num_epochs=200, learning_rate=0.001, n_splits = 5, n_repeats = 5, vocabulary=vocabulary_pnum,
                   patience=11,patience_reduce=5):

  test_predictions = np.zeros((len(X_test),1))
  test_results_df = pd.DataFrame(index=X_test.index, columns=list(range(n_repeats*n_splits)))
  train_results_df = pd.DataFrame(index=X_train.index, columns=list(range(n_repeats*n_splits)))

  all_mse = []
  all_rmse = []

  rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=36851234)

  for i, (train_index, val_index) in enumerate(rskf.split(X_train, X_train["p_num"])):
      print(f"\nRunning CV {i}\n")
      X, val_X = X_train.iloc[train_index], X_train.iloc[val_index]
      y, val_y = X_train[target].iloc[train_index], X_train[target].iloc[val_index]
      X_test = X_test.copy()
      #################################################################### Prepare Datasets loaders:
      train_dataset = dataframe_to_dataset(X, batch_size=batch_size, shuffle=True)
      valid_dataset = dataframe_to_dataset(val_X, batch_size=batch_size, shuffle=False)
      test_dataset = dataframe_to_dataset(X_test, batch_size=batch_size, shuffle=False)

      ##################################################################### Relevant Folders
      folders_experiment = f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/Glucose/{experiment_name}_{i}/"
      ##################################################################### Generate and Fit Model
      # Callbacks:
      checkpoint_filepath = folders_experiment + f'checkpoint/{experiment_name}_{i}.weights.h5'

      # Generate the Model:
      model = model_constructor(vocabulary=vocabulary,
                                lr = learning_rate,
                                **best_params)
      if i>=0:
        print("Start training the model...")

        history = model.fit(train_dataset,
                            epochs=num_epochs,
                            callbacks=[keras.callbacks.EarlyStopping(monitor='val_rmse',
                                                                     patience=patience,
                                                                     mode="min",
                                                                     start_from_epoch=5,
                                                                     restore_best_weights=True),
                                      keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                      save_weights_only=True,
                                                                      monitor="val_rmse",
                                                                      mode='min',
                                                                      save_best_only=True),
                                      keras.callbacks.ReduceLROnPlateau(monitor='val_rmse',
                                                                        factor=0.5,
                                                                        patience=patience_reduce,
                                                                        min_lr=0.0001,
                                                                        mode="min")],
                            validation_data=valid_dataset)

        print("Model training finished")

      model.load_weights(checkpoint_filepath)
      model.evaluate(valid_dataset, verbose=0)

      if i>=0:
        plot_training_session(history)

      oof_res = model.predict(valid_dataset)
      test_pred = model.predict(test_dataset)

      print(f"Out-of-Fold Shapes: {val_y.shape},{oof_res.shape}")

      rmse_score = root_mean_squared_error(val_y, oof_res)


      fig, axs = plt.subplots(1,1, figsize=(10,4))
      axs.scatter(oof_res, val_y)
      axs.set_title(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")
      plt.tight_layout()
      plt.show()

      print(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")

      ##################################################################### Save the Model
      model.save(f"{folders_experiment}/model_{experiment_name}.keras")

      ##################################################################### Create Model Output
      test_results_df.loc[:,i] = test_pred
      all_rmse.append(round(rmse_score, 3))

      #############
      train_results_df.iloc[val_index,i] = oof_res.flatten()

      gc.collect()

    ##################################################################### Create Model Output
  print(f"All Valuation RMSE: {all_rmse}")

  return test_results_df, train_results_df

#### Keras Tuner:

In [ ]:
tuning_on=False
gc.collect()

 Select a Validation set:

In [ ]:
if tuning_on==True:
  Xt, Xv = train_test_split(X_train, test_size=0.20, random_state=42, stratify=X_train['p_num'])

  print(f"Train Shape: {Xt.shape}, Val Shape: {Xv.shape}")

In [ ]:
if tuning_on==True:
  vocabulary = Xt["p_num"].unique().tolist()

  train_dataset = dataframe_to_dataset(Xt, batch_size=512, shuffle=True)
  valid_dataset = dataframe_to_dataset(Xv, batch_size=512, shuffle=False)

In [ ]:
if tuning_on==True:
  # Define the hyperparameter search space: EXPERIMENT 1
  hp = kt.HyperParameters()
  hp.Choice('activation', ["relu","silu","gelu","selu"])
  hp.Float('dropout',0.15,0.40, step=0.025)
#  hp.Float('noise',0.05,0.1, step=0.025)
  hp.Float('reg',0.0001, 1.0,step=10,sampling="log")
  hp.Choice('num_dense_blocks', values=[1,2,3,4])
  hp.Choice('units_dense', values=[128,256,512])

In [ ]:
def create_turner_model(hp):

  model = create_model_v0(vocabulary=vocabulary,
                          dense_layers=[hp.get('units_dense')]+[int(hp.get('units_dense')/2)]*hp.get('num_dense_blocks')+[int(hp.get('units_dense')/4)],
                          dropout=hp.get('dropout'),
                          activation=hp.get('activation'),
                          noise=0, #hp.get('noise'),
                          reg=hp.get('reg'),
                          lr=0.0025)
  return model

In [ ]:
if tuning_on==True:
  # Create a tuner and search for the best hyperparameters
  tuner = BayesianOptimization(create_turner_model,
                              objective=kt.Objective("val_rmse", "min"),
                              hyperparameters=hp, max_trials=50, overwrite=True)

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=7, mode="min", start_from_epoch=5)
  reduce_ = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, min_lr=0.0001, mode="min")

  tuner.search(train_dataset, validation_data=valid_dataset, epochs=16, callbacks=[stop_early,reduce_])


    val_rmse: 1.7364014387130737

    Best val_rmse So Far: 1.736088514328003
    Total elapsed time: 05h 27m 20s

    Search: Running Trial #32

    Value             |Best Value So Far |Hyperparameter
    relu              |silu              |activation
    0.375             |0.35              |dropout
    0.01              |0.001             |reg
    4                 |1                 |num_dense_blocks
    128               |512               |units_dense


In [ ]:
if tuning_on==True:
  tuner.results_summary(num_trials=10)

In [ ]:
if tuning_on==True:
  print(tuner.get_best_hyperparameters(4)[0].values)
  print(tuner.get_best_hyperparameters(4)[1].values)
  print(tuner.get_best_hyperparameters(4)[2].values)
  print(tuner.get_best_hyperparameters(4)[3].values)

* {'activation': 'gelu', 'dropout': 0.15, 'reg': 0.0001, 'num_dense_blocks': 1, 'units_dense': 128}
* {'activation': 'silu', 'dropout': 0.35, 'reg': 0.001, 'num_dense_blocks': 1, 'units_dense': 512}
* {'activation': 'relu', 'dropout': 0.375, 'reg': 0.0001, 'num_dense_blocks': 1, 'units_dense': 512}
* {'activation': 'relu', 'dropout': 0.175, 'reg': 0.1, 'num_dense_blocks': 3, 'units_dense': 128}

* Trial 47 summary
Hyperparameters:
activation: gelu
dropout: 0.15
reg: 0.0001
num_dense_blocks: 1
units_dense: 128
Score: 1.727567195892334

* Trial 11 summary
Hyperparameters:
activation: silu
dropout: 0.35
reg: 0.001
num_dense_blocks: 1
units_dense: 512
Score: 1.736088514328003

* Trial 35 summary
Hyperparameters:
activation: relu
dropout: 0.375
reg: 0.0001
num_dense_blocks: 1
units_dense: 512
Score: 1.7361124753952026

* Trial 13 summary
Hyperparameters:
activation: relu
dropout: 0.175
reg: 0.1
num_dense_blocks: 3
units_dense: 128
Score: 1.7361220121383667

* Trial 14 summary
Hyperparameters:
activation: gelu
dropout: 0.275
reg: 0.001
num_dense_blocks: 1
units_dense: 512
Score: 1.7361259460449219

* Trial 26 summary
Hyperparameters:
activation: selu
dropout: 0.375
reg: 0.0001
num_dense_blocks: 4
units_dense: 128
Score: 1.7361326217651367

#### Fit The Model:

In [ ]:
#del df_train_scaled #Xt, Xv, train_dataset, valid_dataset, stop_early, reduce_#, tuner
gc.collect()

In [ ]:
bas = 512
rep=1
[bas]+[int(bas/2)]*rep+[int(bas/4)]

In [ ]:
#best_params = tuner.get_best_hyperparameters(1)[0].values
best_params = {'activation': 'gelu', 'dropout': 0.15, 'reg': 0.0001, 'dense_layers': [128,64,32]}

In [ ]:
test_results_df, train_results_df = run_experiment(X_train, X_test, create_model_v0, best_params, experiment_name = "baseline_nn_ext_v2_st",
                                                  rs=42, target="bg+1:00", batch_size=512, num_epochs=51, learning_rate=0.0005,
                                                  n_splits = 3, n_repeats = 3, vocabulary = vocabulary_pnum)

All Valuation RMSE: [1.71, 1.701, 1.715, 1.716, 1.708, 1.702, 1.701, 1.708, 1.708]

In [ ]:
test_results_df["average"] = test_results_df.mean(axis=1)
test_results_df

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(10,4))
axs = np.ravel(axs)

axs[0].scatter(test_results_df["average"], test_results_df[0])
axs[0].set_title("Average Predictions vs. Val 0")

axs[1].scatter(test_results_df["average"], test_results_df[1])
axs[1].set_title("Average Predictions vs. Val 1")

axs[2].scatter(test_results_df[1], test_results_df[2])
axs[2].set_title("Val 1 Predictions vs. Val 2")

axs[3].scatter(test_results_df[2], test_results_df[8])
axs[3].set_title("Val 2 Predictions vs. Val 8")

plt.tight_layout()
plt.show()

#### **STORE RESULTS**

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")

In [ ]:
for i, col in enumerate(test_results_df.columns):
  sub["bg+1:00"] = test_results_df[col].values
  sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v2_{col}_expanded_st.csv", index=False)
  print(sub.isna().sum())

In [ ]:
train_results_df.head(10)

In [ ]:
train_results_df["average"] = train_results_df.mean(axis=1)
train_results_df.isna().sum()

In [ ]:
train_results_df_ = train_results_df["average"].astype("float").copy()
train_results_df_.columns = ["bg+1:00_v0_expanded"]
train_results_df_.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/train_results_v2_expanded_st.csv.csv")

## **2.2 MODEL 02**

In [ ]:
##Dataset extentions

colx = ['steps_av0','steps_av1','steps_av2','steps_av3','steps_av4','steps_av5','steps_av6','steps_av7','steps_av8','steps_av9','steps_av10','steps_av11','steps_av12','steps_av13','steps_av14',
        'steps_av15','steps_av16','steps_av17','steps_av18','steps_av19','steps_av20','steps_av21','steps_av22','steps_av23','steps_av24','steps_av25','steps_av26','steps_av27','steps_av28','steps_av29','steps_av30','steps_av31',
        'insulin_av0','insulin_av1','insulin_av2','insulin_av3','insulin_av4','insulin_av5','insulin_av6','insulin_av7','insulin_av8','insulin_av9','insulin_av10','insulin_av11',
        'insulin_av12','insulin_av13','insulin_av14','insulin_av15','insulin_av16','insulin_av17','insulin_av18','insulin_av19','insulin_av20','insulin_av21','insulin_av22','insulin_av23','insulin_av24','insulin_av25','insulin_av26',
        'insulin_av27','insulin_av28','insulin_av29','insulin_av30','insulin_av31','activity0','activity1','activity2','activity3','activity4','activity5','activity6','activity7','activity8','activity9','activity10',
        'activity11','activity12','activity13','activity14','activity15','activity16','activity17','activity18','activity19','activity20','activity21','activity22','activity23','activity24','activity25',
        'activity26','activity27','activity28','activity29','activity30','activity31','cals_av0','cals_av1','cals_av2','cals_av3','cals_av4','cals_av5','cals_av6','cals_av7','cals_av8','cals_av9',
        'cals_av10','cals_av11','cals_av12','cals_av13','cals_av14','cals_av15','cals_av16','cals_av17','cals_av18','cals_av19','cals_av20','cals_av21','cals_av22','cals_av23','cals_av24','cals_av25','cals_av26','cals_av27','cals_av28','cals_av29','cals_av30','cals_av31',
        'brake0','brake1','brake2','brake3','brake4','brake5','brake6','brake7','brake8','brake9','brake10','brake11','brake12','brake13',
        'brake14','brake15','brake16','brake17','brake18','brake19','brake20','brake21','brake22','brake23','brake24','brake25','brake26','brake27','brake28','brake29','brake30','brake31',
        'carbs_av0','carbs_av1','carbs_av2','carbs_av3','carbs_av4','carbs_av5','carbs_av6','carbs_av7','carbs_av8','carbs_av9','carbs_av10','carbs_av11','carbs_av12','carbs_av13','carbs_av14','carbs_av15','carbs_av16',
        'carbs_av17','carbs_av18','carbs_av19','carbs_av20','carbs_av21','carbs_av22','carbs_av23','carbs_av24','carbs_av25','carbs_av26','carbs_av27','carbs_av28','carbs_av29','carbs_av30','carbs_av31',
        'bg0','bg1','bg2','bg3','bg4','bg5','bg6','bg7','bg8','bg9','bg10','bg11','bg12','bg13','bg14','bg15','bg16','bg17','bg18','bg19','bg20','bg21','bg22','bg23','bg24','bg25','bg26','bg27','bg28','bg29','bg30','bg31',
        'hr0','hr1','hr2','hr3','hr4','hr5','hr6','hr7','hr8','hr9','hr10','hr11','hr12','hr13','hr14','hr15','hr16','hr17','hr18','hr19','hr20','hr21','hr22','hr23','hr24','hr25','hr26','hr27','hr28','hr29','hr30','hr31',
        'intake0','intake1','intake2','intake3','intake4','intake5','intake6','intake7','intake8','intake9','intake10','intake11','intake12','intake13','intake14',
        'intake15','intake16','intake17','intake18','intake19','intake20','intake21','intake22','intake23','intake24','intake25','intake26','intake27','intake28','intake29','intake30','intake31']

usecols = ['bg0','bg1','bg2','bg3','bg4','bg5','bg6','bg7','bg8','bg9','bg10','bg11','bg12','bg13','bg14','bg15','bg16','bg17','bg18','bg19','bg20','bg21','bg22','bg23','bg24','bg25','bg26','bg27','bg28','bg29','bg30','bg31',
           'insulin_av0','insulin_av1','insulin_av2','insulin_av3','insulin_av4','insulin_av5','insulin_av6','insulin_av7','insulin_av8','insulin_av9','insulin_av10','insulin_av11','insulin_av12','insulin_av13','insulin_av14',
           'insulin_av15','insulin_av16','insulin_av17','insulin_av18','insulin_av19','insulin_av20','insulin_av21','insulin_av22','insulin_av23','insulin_av24','insulin_av25','insulin_av26', 'insulin_av27','insulin_av28',
           'insulin_av29','insulin_av30','insulin_av31','brake0','brake1','brake2','brake3','brake4','brake5','brake6','brake7','brake8','brake9','brake10','brake11','brake12','brake13', 'brake14','brake15','brake16','brake17',
           'brake18','brake19','brake20','brake21','brake22','brake23','brake24','brake25','brake26','brake27','brake28','brake29','brake30','brake31','intake0','intake1','intake2','intake3','intake4','intake5','intake6','intake7',
           'intake8','intake9','intake10','intake11','intake12','intake13','intake14','intake15','intake16','intake17','intake18','intake19','intake20','intake21','intake22','intake23','intake24','intake25','intake26','intake27','intake28','intake29','intake30','intake31']

dict_colx = {i:np.float32 for i in colx}

df_train_ext=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled_smaller.csv", dtype=dict_colx, usecols=usecols)
df_test_ext=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_test_scaled_smaller.csv", dtype=dict_colx, usecols=usecols)

df_test_ext.shape

In [ ]:
df_test_ext.index = X_test.index
#X_test.head(3)

In [ ]:
len(usecols)
X_train.shape

In [ ]:
df_train_ext.head(3)

In [ ]:
X_train.shape,df_train_ext.shape

In [ ]:
scaler_ext = StandardScaler()
df_train_ext[usecols] = scaler_ext.fit_transform(df_train_ext[usecols])
df_test_ext[usecols] = scaler_ext.transform(df_test_ext[usecols])

In [ ]:
df_train_ext.max().max(),df_train_ext.min().min(),df_train_ext["bg18"].mean()

In [ ]:
df_train_ext.iloc[:,-55:-45].describe()

In [ ]:
X_train = pd.concat([df_train_ext,X_train],axis=1)
X_test = pd.concat([df_test_ext,X_test],axis=1)
X_train.shape,X_test.shape

In [ ]:
gc.collect()

In [ ]:
X_train["diff"] = X_train["bg31"]-X_train["bg0"]
X_test["diff"] = X_test["bg31"]-X_test["bg0"]

scaler_diff = StandardScaler()
X_train["diff"] = scaler_diff.fit_transform(X_train[["diff"]])
X_test["diff"] = scaler_diff.transform(X_test[["diff"]])

In [ ]:
X_train[["diff","bg31","insulin_av31","bg+1:00",'PC_1', 'PC_2', 'PC_3', 'PC_4', 'enc_08_v6', 'enc_07_v7', 'enc_03_v7', 'enc_01_v5',
              'enc_02_v5', 'enc_03_v5', 'enc_04_v5', 'enc_05_v5', 'enc_06_v5', 'enc_07_v5', 'enc_08_v5']].corr()

In [ ]:
# feat = "insulin_av"
# for i_ in range(31):
#   ver = X_train[[f"{feat}{i_}",f"{feat}31","bg+1:00"]].copy()
#   ver["diff"] = ver[f"{feat}31"]-ver[f"{feat}{i_}"]
#   print("{}{} - Corr diff: {} - Corr Level: {}".format(feat,i_,np.round(ver["diff"].corr(ver["bg+1:00"]), 3),np.round(ver[f"{feat}{i_}"].corr(ver["bg+1:00"]), 3)))

In [ ]:
X_train = X_train[["diff","bg31","insulin_av31","bg+1:00",'p_num','PC_1', 'PC_2', 'PC_3', 'PC_4', 'enc_08_v6',
                   'enc_07_v7', 'enc_03_v7', 'enc_01_v5','enc_02_v5', 'enc_03_v5', 'enc_04_v5', 'enc_05_v5',
                   'enc_06_v5', 'enc_07_v5', 'enc_08_v5']]

X_test = X_test[["diff","bg31","insulin_av31","bg+1:00",'p_num','PC_1', 'PC_2', 'PC_3', 'PC_4', 'enc_08_v6',
                   'enc_07_v7', 'enc_03_v7', 'enc_01_v5','enc_02_v5', 'enc_03_v5', 'enc_04_v5', 'enc_05_v5',
                   'enc_06_v5', 'enc_07_v5', 'enc_08_v5']]

X_train.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_ext_new.csv")
X_test.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_test_ext_new.csv")

# X_train=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_ext_new.csv")
# X_test=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_test_ext_new.csv", index_col=0)

In [ ]:
X_test.head(3)

### 2.1.1 Create Dataloader:

In [ ]:
gc.collect()

In [ ]:
def dataframe_to_dataset(dataframe_, shuffle=False, batch_size=32):
    dataframe = dataframe_.copy()
    target = dataframe["bg+1:00"]
    dataframe = dataframe.drop(columns=["bg+1:00"])
    tab_data = dataframe.drop(columns=["p_num"])

    ds = tf.data.Dataset.from_tensor_slices(((dataframe["p_num"].values,  # First input
                                              tab_data.values), # Second input
                                              target))

    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [ ]:
train_ds = dataframe_to_dataset(X_train,batch_size=1)

* **TEST THE DATALOADER:**

In [ ]:
for (x0, x1), y in train_ds.take(1):
    print(x0.shape,x1.shape,y.shape)

In [ ]:
#X_train.iloc[:2000,:].to_csv("train_sample.csv")

### **2.1.3 Create Models:**

In [ ]:
X_train.info()

In [ ]:
def create_model_v0(vocabulary=vocabulary_pnum,
                    dense_layers=[256,128,128,64],
                    dropout=0.3,
                    activation="selu",
                    reg=0.0,
                    lr=0.001,
                    noise=0.05,
                    summary=False):

  # --- Tabular Data Processing ---
  # Input layer for tabular data as a dictionary

  pnum_input = keras.Input(shape=(1,), name="pnum_input", dtype="string")

  # Lookup Layer for the p_num:
  lookup = layers.StringLookup(vocabulary=vocabulary,mask_token=None,num_oov_indices=1,
                output_mode="int", name="lookup_pnum")
  pnum_encoded = lookup(pnum_input)

  # Embedding layers for hour and minute
  pnum_encoded_embedding = layers.Embedding(input_dim=len(vocabulary)+1,
                                            output_dim=5,
                                            name="embed_pnum")(pnum_encoded)
  # Flatten the embeddings
  pnum_flat = layers.Flatten()(pnum_encoded_embedding)

  # Input layer for Tabular Data
  tab_data = keras.Input(shape=(18,), name="tab_input")

  # Concatenate all tabular features
  x = layers.concatenate([tab_data, pnum_flat])
  x = layers.BatchNormalization()(x)

  x = layers.GaussianNoise(stddev=noise, seed=42, name="gn_00")(x)
  tabular_output = x
  tabular_output = layers.Dropout(dropout, name="bb_00")(tabular_output)

  # Dense layers for tabular data
  for num, den in enumerate(dense_layers):
    tabular_output = dense_block(den,dropout=dropout,activation=activation, reg=reg, name=f"block_{num}")(tabular_output)


  tabular_output = layers.concatenate([x, tabular_output])
  # --- Output Layer ---
  # Final dense layer for prediction
  output = layers.Dense(1, name="output")(tabular_output)

  # --- Create and Compile the Model ---
  # Create the model
  model = keras.Model(inputs=[pnum_input, tab_data], outputs=output)
  optimizer= keras.optimizers.Adam(learning_rate=lr)
  metric = RootMeanSquaredError(name="rmse", dtype=None)
  metric_mae = MeanAbsoluteError(name="mae", dtype=None)

  # Compile the model
  model.compile(optimizer=optimizer, loss="mse", metrics=[metric,metric_mae])

  # Print model summary
  if summary==True:
    model.summary()

  return model

model = create_model_v0(vocabulary=vocabulary_pnum,summary=True)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True, rankdir="LR", expand_nested=True, show_layer_activations=True)

### 2.1.4 Training Functions:

In [ ]:
list(range(len(val_sets)))

#### Main Function:

In [ ]:
def run_experiment(X_train, X_test, model_constructor, best_params, experiment_name="baseline_nn_ext_v1", rs=42, target="bg+1:00",
                   batch_size=64, num_epochs=200, learning_rate=0.001, n_splits = 5, n_repeats = 5, vocabulary=vocabulary_pnum,
                   patience=11,patience_reduce=5):

  test_predictions = np.zeros((len(X_test),1))
  test_results_df = pd.DataFrame(index=X_test.index, columns=list(range(n_repeats*n_splits)))
  train_results_df = pd.DataFrame(index=X_train.index, columns=list(range(n_repeats*n_splits)))

  all_mse = []
  all_rmse = []

  rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=36851234)

  for i, (train_index, val_index) in enumerate(rskf.split(X_train, X_train["p_num"])):
      print(f"\nRunning CV {i}\n")
      X, val_X = X_train.iloc[train_index], X_train.iloc[val_index]
      y, val_y = X_train[target].iloc[train_index], X_train[target].iloc[val_index]
      X_test = X_test.copy()
      #################################################################### Prepare Datasets loaders:
      train_dataset = dataframe_to_dataset(X, batch_size=batch_size, shuffle=True)
      valid_dataset = dataframe_to_dataset(val_X, batch_size=batch_size, shuffle=False)
      test_dataset = dataframe_to_dataset(X_test, batch_size=batch_size, shuffle=False)

      ##################################################################### Relevant Folders
      folders_experiment = f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/Glucose/{experiment_name}_{i}/"
      ##################################################################### Generate and Fit Model
      # Callbacks:
      checkpoint_filepath = folders_experiment + f'checkpoint/{experiment_name}_{i}.weights.h5'

      # Generate the Model:
      model = model_constructor(vocabulary=vocabulary,
                                lr = learning_rate,
                                **best_params)
      if i>=0:
        print("Start training the model...")

        history = model.fit(train_dataset,
                            epochs=num_epochs,
                            callbacks=[keras.callbacks.EarlyStopping(monitor='val_rmse',
                                                                     patience=patience,
                                                                     mode="min",
                                                                     start_from_epoch=5,
                                                                     restore_best_weights=True),
                                      keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                      save_weights_only=True,
                                                                      monitor="val_rmse",
                                                                      mode='min',
                                                                      save_best_only=True),
                                      keras.callbacks.ReduceLROnPlateau(monitor='val_rmse',
                                                                        factor=0.5,
                                                                        patience=patience_reduce,
                                                                        min_lr=0.0001,
                                                                        mode="min")],
                            validation_data=valid_dataset)

        print("Model training finished")

      model.load_weights(checkpoint_filepath)
      model.evaluate(valid_dataset, verbose=0)

      if i>=0:
        plot_training_session(history)

      oof_res = model.predict(valid_dataset)
      test_pred = model.predict(test_dataset)

      print(f"Out-of-Fold Shapes: {val_y.shape},{oof_res.shape}")

      rmse_score = root_mean_squared_error(val_y, oof_res)


      fig, axs = plt.subplots(1,1, figsize=(10,4))
      axs.scatter(oof_res, val_y)
      axs.set_title(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")
      plt.tight_layout()
      plt.show()

      print(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")

      ##################################################################### Save the Model
      model.save(f"{folders_experiment}/model_{experiment_name}.keras")

      ##################################################################### Create Model Output
      test_results_df.loc[:,i] = test_pred
      all_rmse.append(round(rmse_score, 3))

      #############
      train_results_df.iloc[val_index,i] = oof_res.flatten()

      gc.collect()

    ##################################################################### Create Model Output
  print(f"All Valuation RMSE: {all_rmse}")

  return test_results_df, train_results_df

#### Keras Tuner:

In [ ]:
tuning_on=False
gc.collect()

 Select a Validation set:

In [ ]:
if tuning_on==True:
  Xt, Xv = train_test_split(X_train, test_size=0.20, random_state=42, stratify=X_train['p_num'])

  print(f"Train Shape: {Xt.shape}, Val Shape: {Xv.shape}")

In [ ]:
if tuning_on==True:
  vocabulary = Xt["p_num"].unique().tolist()

  train_dataset = dataframe_to_dataset(Xt, batch_size=512, shuffle=True)
  valid_dataset = dataframe_to_dataset(Xv, batch_size=512, shuffle=False)

In [ ]:
if tuning_on==True:
  # Define the hyperparameter search space: EXPERIMENT 1
  hp = kt.HyperParameters()
  hp.Choice('activation', ["relu","silu","gelu","selu"])
  hp.Float('dropout',0.15,0.40, step=0.025)
#  hp.Float('noise',0.05,0.1, step=0.025)
  hp.Float('reg',0.0001, 1.0,step=10,sampling="log")
  hp.Choice('num_dense_blocks', values=[1,2,3,4])
  hp.Choice('units_dense', values=[128,256,512])

In [ ]:
def create_turner_model(hp):

  model = create_model_v0(vocabulary=vocabulary,
                          dense_layers=[hp.get('units_dense')]+[int(hp.get('units_dense')/2)]*hp.get('num_dense_blocks')+[int(hp.get('units_dense')/4)],
                          dropout=hp.get('dropout'),
                          activation=hp.get('activation'),
                          noise=0, #hp.get('noise'),
                          reg=hp.get('reg'),
                          lr=0.0025)
  return model

In [ ]:
if tuning_on==True:
  # Create a tuner and search for the best hyperparameters
  tuner = BayesianOptimization(create_turner_model,
                              objective=kt.Objective("val_rmse", "min"),
                              hyperparameters=hp, max_trials=50, overwrite=True)

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=7, mode="min", start_from_epoch=5)
  reduce_ = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, min_lr=0.0001, mode="min")

  tuner.search(train_dataset, validation_data=valid_dataset, epochs=16, callbacks=[stop_early,reduce_])


    val_rmse: 1.7364014387130737

    Best val_rmse So Far: 1.736088514328003
    Total elapsed time: 05h 27m 20s

    Search: Running Trial #32

    Value             |Best Value So Far |Hyperparameter
    relu              |silu              |activation
    0.375             |0.35              |dropout
    0.01              |0.001             |reg
    4                 |1                 |num_dense_blocks
    128               |512               |units_dense


In [ ]:
if tuning_on==True:
  tuner.results_summary(num_trials=10)

In [ ]:
if tuning_on==True:
  print(tuner.get_best_hyperparameters(4)[0].values)
  print(tuner.get_best_hyperparameters(4)[1].values)
  print(tuner.get_best_hyperparameters(4)[2].values)
  print(tuner.get_best_hyperparameters(4)[3].values)

* {'activation': 'gelu', 'dropout': 0.15, 'reg': 0.0001, 'num_dense_blocks': 1, 'units_dense': 128}
* {'activation': 'silu', 'dropout': 0.35, 'reg': 0.001, 'num_dense_blocks': 1, 'units_dense': 512}
* {'activation': 'relu', 'dropout': 0.375, 'reg': 0.0001, 'num_dense_blocks': 1, 'units_dense': 512}
* {'activation': 'relu', 'dropout': 0.175, 'reg': 0.1, 'num_dense_blocks': 3, 'units_dense': 128}

* Trial 47 summary
Hyperparameters:
activation: gelu
dropout: 0.15
reg: 0.0001
num_dense_blocks: 1
units_dense: 128
Score: 1.727567195892334

* Trial 11 summary
Hyperparameters:
activation: silu
dropout: 0.35
reg: 0.001
num_dense_blocks: 1
units_dense: 512
Score: 1.736088514328003

* Trial 35 summary
Hyperparameters:
activation: relu
dropout: 0.375
reg: 0.0001
num_dense_blocks: 1
units_dense: 512
Score: 1.7361124753952026

* Trial 13 summary
Hyperparameters:
activation: relu
dropout: 0.175
reg: 0.1
num_dense_blocks: 3
units_dense: 128
Score: 1.7361220121383667

* Trial 14 summary
Hyperparameters:
activation: gelu
dropout: 0.275
reg: 0.001
num_dense_blocks: 1
units_dense: 512
Score: 1.7361259460449219

* Trial 26 summary
Hyperparameters:
activation: selu
dropout: 0.375
reg: 0.0001
num_dense_blocks: 4
units_dense: 128
Score: 1.7361326217651367

#### Fit The Model:

In [ ]:
#del df_train_scaled #Xt, Xv, train_dataset, valid_dataset, stop_early, reduce_#, tuner
gc.collect()

In [ ]:
bas = 512
rep=1
[bas]+[int(bas/2)]*rep+[int(bas/4)]

In [ ]:
#best_params = tuner.get_best_hyperparameters(1)[0].values
best_params = {'activation': 'gelu', 'dropout': 0.15, 'reg': 0.0001, 'dense_layers': [128,64,32]}

In [ ]:
test_results_df, train_results_df = run_experiment(X_train, X_test, create_model_v0, best_params, experiment_name = "baseline_nn_extra_v3_st",
                                                  rs=42, target="bg+1:00", batch_size=512, num_epochs=51, learning_rate=0.0005,
                                                  n_splits = 3, n_repeats = 3, vocabulary = vocabulary_pnum)

All Valuation RMSE: [2.195, 1.99, 2.329, 0.841]

In [ ]:
test_results_df["average"] = test_results_df.mean(axis=1)
test_results_df

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(10,4))
axs = np.ravel(axs)

axs[0].scatter(test_results_df["average"], test_results_df[0])
axs[0].set_title("Average Predictions vs. Val 0")

axs[1].scatter(test_results_df["average"], test_results_df[1])
axs[1].set_title("Average Predictions vs. Val 1")

axs[2].scatter(test_results_df[1], test_results_df[2])
axs[2].set_title("Val 1 Predictions vs. Val 2")

axs[3].scatter(test_results_df[2], test_results_df[8])
axs[3].set_title("Val 2 Predictions vs. Val 8")

plt.tight_layout()
plt.show()

#### **STORE RESULTS**

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")

In [ ]:
for i, col in enumerate(test_results_df.columns):
  sub["bg+1:00"] = test_results_df[col].values
  sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v3_{col}_expanded_st.csv", index=False)
  print(sub.isna().sum())

In [ ]:
train_results_df.head(10)

In [ ]:
train_results_df["average"] = train_results_df.mean(axis=1)
train_results_df.isna().sum()

In [ ]:
train_results_df_ = train_results_df["average"].astype("float").copy()
train_results_df_.columns = ["bg+1:00_v0_expanded"]
train_results_df_.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/train_results_v3_expanded_st.csv.csv")

## **2.3 MODEL 03**

### 3.1.1 Create Dataloader:

In [ ]:
gc.collect()

In [ ]:
def dataframe_to_dataset(dataframe, shuffle=False, batch_size=32, flds = ts_fields):
    dataframe = dataframe.copy()
    target = dataframe["bg+1:00"]
    dataframe = dataframe.drop(columns=["bg+1:00"])

    timeseries_df = dataframe[flds].values
    timeseries_df = timeseries_df.reshape((-1, 72, 8))
    static_df = dataframe.drop(columns=flds)


    ds = tf.data.Dataset.from_tensor_slices(((static_df["hour"].values,  # First input
                                              static_df["minute"].values,  # Second input
                                              static_df["cluster"].values,  # Third input
                                              static_df["cluster_pca"].values,  # Fourth input
                                              static_df[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4","enc_05_v4",
                                                        "enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7","enc_08_v6","enc_04_v6"]].values,  # Fourth input
                                              timeseries_df),
                                              target))

    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [ ]:
train_ds = dataframe_to_dataset(X_train,batch_size=1)

* **TEST THE DATALOADER:**

In [ ]:
for (x0, x1, x2, x3, x4, x5), y in train_ds.take(1):
    print(x0.shape,x1.shape,x2.shape,x3.shape,x4.shape,x5.shape,y.shape)

In [ ]:
#X_train.iloc[:2000,:].to_csv("train_sample.csv")

In [ ]:
X_train.info()

### **2.1.3 Create Models:**

In [ ]:
def residual_block(input_tensor, filters, kernel_size, strides=1):
    x = layers.Conv1D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = layers.LayerNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.SpatialDropout1D(0.2)(x)

    # Residual connection
    if strides > 1 or input_tensor.shape[-1] != filters:
        residual = layers.Conv1D(filters, 1, strides=strides, padding='same')(input_tensor)
    else:
        residual = input_tensor

    x = layers.add([x, residual])
    x = layers.MaxPooling1D(pool_size=2)(x)
    return x

In [ ]:
def create_model_v2(dense_layers=[256,128,128,128,64],
                    dropout=0.3, conv_layers=[1024,512,256,128],
                    activation="selu", reg=0.0,lr=0.001,strides=1,kernel_size=3, summary=False):

  # --- Tabular Data Processing ---
  # Input layer for tabular data as a dictionary
  hour_input = keras.Input(shape=(1,), name="hour_input")
  minute_input = keras.Input(shape=(1,), name="minute_input")
  cluster_input = keras.Input(shape=(1,), name="cluster_input")
  pca_input = keras.Input(shape=(1,), name="cluster_pca")
  cont_input = keras.Input(shape=(15,), name="cont_inputs")

  # Lookup Layer for the p_num:
  # pca_lookup = layers.IntegerLookup(vocabulary=list(range(0,4)),mask_token=None,num_oov_indices=0,
  #                                  output_mode="int", name="lookup_pca")
  # pca_encoded = pca_lookup(pca_input)

  # Lookup Layer for the minutes:
  minute_lookup = layers.IntegerLookup(vocabulary=list(range(0,60,5)),mask_token=None,num_oov_indices=0,
                                       output_mode="int", name="lookup_min")

  minute_encoded = minute_lookup(minute_input)

  # Embedding layers for hour and minute
  hour_embedding = layers.Embedding(input_dim=24, output_dim=8, name="embed_hour")(hour_input)
  minute_embedding = layers.Embedding(input_dim=12, output_dim=6, name="embed_minute")(minute_encoded)
  pca_encoded_embedding = layers.Embedding(input_dim=4,output_dim=3, name="embed_pca")(pca_input)

  # Embedding layer for cluster
  cluster_embedding = layers.Embedding(input_dim=22, output_dim=8, name="embed_cluster")(cluster_input)

  # Flatten the embeddings
  hour_flat = layers.Flatten()(hour_embedding)
  minute_flat = layers.Flatten()(minute_embedding)
  cluster_flat = layers.Flatten()(cluster_embedding)
  pca_flat = layers.Flatten()(pca_encoded_embedding)

  # Concatenate all tabular features
  tabular_output = layers.concatenate([hour_flat, minute_flat, cluster_flat, pca_flat,cont_input],
                                      name="tabular_concat")

  # Dense layers for tabular data
  for num, den in enumerate(dense_layers):
    tabular_output = residual_block_dense(tabular_output, den, activation=activation, normalization_type='batch', dropout_rate=dropout, name=f"block_{num}",  reg=reg,)

  # --- Time Series Data Processing ---
  # Input layer for time series data
  time_series_input = keras.Input(shape=(72, 8), name="time_series_input")
  x=time_series_input
  # LSTM layers for time series data
  for sup, i in enumerate(conv_layers):
      input_tensor = x
      x = layers.Conv1D(filters=i, kernel_size=3, strides=strides, padding='same',name=f"conv_{sup}")(x)
      x = layers.LayerNormalization(name=f"lnorm_{sup}")(x)
      x = layers.Activation('relu',name=f"act_{sup}")(x)
      x = layers.SpatialDropout1D(0.2,name=f"spat_drop_{sup}")(x)

      # Residual connection
      if strides > 1 or time_series_input.shape[-1] != i:
          residual = layers.Conv1D(i, 1, strides=strides, padding='same',name=f"res_conn_{sup}")(input_tensor)
      else:
          residual = input_tensor

      x = layers.add([x, residual],name=f"add_{sup}")
      x = layers.MaxPooling1D(pool_size=2,name=f"max_pool_{sup}")(x)

  x = layers.Flatten(name=f"flatten_conv")(x)
  # --- Combine Tabular and Time Series Data ---
  # Concatenate the outputs from both branches
  concatenated = layers.concatenate([tabular_output, x])

  # --- Output Layer ---
  # Final dense layer for prediction
  output = layers.Dense(1,
                        name="output",
                        #activity_regularizer=keras.regularizers.l2(reg)
                        )(concatenated)

  # --- Create and Compile the Model ---
  # Create the model
  model = keras.Model(inputs=[hour_input, minute_input, cluster_input, pca_input,cont_input,time_series_input], outputs=output)

  optimizer= keras.optimizers.Adam(learning_rate=lr)
  metric = RootMeanSquaredError(name="rmse", dtype=None)
  metric_mae = MeanAbsoluteError(name="mae", dtype=None)

  # Compile the model
  model.compile(optimizer=optimizer, loss="mse", metrics=[metric,metric_mae])

  # Print model summary
  if summary==True:
    model.summary()

  return model


#vocabulary = X_train["p_num"].unique().tolist()
model = create_model_v2(summary=True)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True, rankdir="LR", expand_nested=True, show_layer_activations=True)

### 2.1.4 Training Functions:

In [ ]:
list(range(len(val_sets)))

#### Main Function:

In [ ]:
def run_experiment(X_train, X_test, model_constructor, best_params, split=val_sets, experiment_name="conv_v0_nn", rs=42, target="bg+1:00",
                   batch_size=64, num_epochs=200, learning_rate=0.001, target_scaler=target_scaler):

  test_predictions = np.zeros((len(X_test),1))
  test_results_df = pd.DataFrame(index=X_test.index, columns=list(range(len(val_sets))))

  all_mse = []
  all_rmse = []

  for i, val_index in enumerate(split):

    print(f"\nRunning CV {i}\n")
    ########################################################################## Prepare the Dataset:
    X_trn = X_train.drop(index=val_index)
    X_val = X_train.loc[val_index,:]

#    vocabulary = X_trn["p_num"].unique().tolist()

    X = X_trn.drop(columns=[target]).copy()
    y = X_trn[target].copy()

    val_X = X_val.drop(columns=[target]).copy()
    val_y = X_val[target].copy()

    X_test = X_test.copy()
    #################################################################### Prepare Datasets loaders:

    train_dataset = dataframe_to_dataset(X_trn, batch_size=batch_size, shuffle=True)
    valid_dataset = dataframe_to_dataset(X_val, batch_size=batch_size, shuffle=False)
    test_dataset = dataframe_to_dataset(X_test, batch_size=batch_size, shuffle=False)

    ##################################################################### Relevant Folders
    folders_experiment = f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/Glucose/{experiment_name}_{i}/"
    ##################################################################### Generate and Fit Model
    # Callbacks:
    checkpoint_filepath = folders_experiment + f'checkpoint/{experiment_name}.weights.h5'

    # Generate the Model:
    model = model_constructor(lr = learning_rate,
                              **best_params)

    print("Start training the model...")
    history = model.fit(train_dataset,
                        epochs=num_epochs,
                        callbacks=[keras.callbacks.EarlyStopping(monitor='val_rmse', patience=17, mode="min",
                                                  start_from_epoch=5,restore_best_weights=True),
                                   keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                    save_weights_only=True,
                                                    monitor="val_rmse",
                                                    mode='min',
                                                    save_best_only=True),
                                   keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5,
                                                          patience=5, min_lr=0.0001, mode="min")],
                        validation_data=valid_dataset)
    print("Model training finished")

    model.load_weights(checkpoint_filepath)
    model.evaluate(valid_dataset, verbose=0)

    plot_training_session(history)

    oof_res = model.predict(valid_dataset)
    test_pred = model.predict(test_dataset)

    print(f"Out-of-Fold Shapes: {val_y.shape},{oof_res.shape}")

    rmse_score = root_mean_squared_error(val_y, oof_res)

    val_pred = target_scaler.inverse_transform(oof_res)
    real_val_y = target_scaler.inverse_transform(val_y.values.reshape(-1,1))
    test_pred_real = target_scaler.inverse_transform(test_pred)

    rmse_score_original = root_mean_squared_error(real_val_y, val_pred)

    fig, axs = plt.subplots(1,1, figsize=(10,4))
    axs.scatter(val_pred, real_val_y)
    axs.set_title(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")
    plt.tight_layout()
    plt.show()

    print(f"Out-of-Fold RMSE Score Rebased: {round(rmse_score, 3)}%")
    print(f"Out-of-Fold RMSE Score Original: {round(rmse_score_original, 3)}%")

    ##################################################################### Save the Model
    model.save(f"{folders_experiment}/model_{experiment_name}.keras")

    ##################################################################### Create Model Output
    test_results_df.loc[:,i] = test_pred_real
    all_rmse.append(round(rmse_score_original, 3))

    gc.collect()

  ##################################################################### Create Model Output
  print(f"All Valuation RMSE: {all_rmse}")

  return test_results_df

#### Keras Tuner:

In [ ]:
tuning_on=True
gc.collect()

 Select a Validation set:

In [ ]:
valid_index = val_sets[0]
Xt = X_train.drop(index=valid_index)
Xv = X_train.loc[valid_index,:]

print(f"Train Shape: {Xt.shape}, Val Shape: {Xv.shape}")

In [ ]:
if tuning_on==True:

  train_dataset = dataframe_to_dataset(Xt, batch_size=256, shuffle=True)
  valid_dataset = dataframe_to_dataset(Xv, batch_size=256, shuffle=False)

In [ ]:
512/4

In [ ]:
if tuning_on==True:
  # Define the hyperparameter search space: EXPERIMENT 1
  hp = kt.HyperParameters()
  hp.Choice('main_activation', ["relu","silu","gelu","selu"]) #
  hp.Float('dropout',0.15,0.40, step=0.025) #
  hp.Float('reg',0.0001, 1.0,step=10,sampling="log") #
  hp.Choice('dense_layers', values=[512,256,128])
  hp.Choice('conv_layers', values=[1024,512,256,128])

In [ ]:
def create_turner_model(hp):

  model = create_model_v2(dense_layers=[hp.get('dense_layers'),int(hp.get('dense_layers')/2),int(hp.get('dense_layers')/2),int(hp.get('dense_layers')/2),int(hp.get('dense_layers')/4)],
                          dropout=hp.get('dropout'),
                          conv_layers=[hp.get('conv_layers'),int(hp.get('conv_layers')/2),int(hp.get('conv_layers')/4),int(hp.get('conv_layers')/8)],
                          activation=hp.get('main_activation'),
                          reg=hp.get('reg'),
                          lr=0.001,
                          summary=False)
  return model

In [ ]:
if tuning_on==True:
  # Create a tuner and search for the best hyperparameters
  tuner = BayesianOptimization(create_turner_model,
                              objective=kt.Objective("val_rmse", "min"),
                              hyperparameters=hp, max_trials=50, overwrite=True)

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=7, mode="min", start_from_epoch=5)
  reduce_ = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, min_lr=0.0001, mode="min")

  tuner.search(train_dataset, validation_data=valid_dataset, epochs=31, callbacks=[stop_early,reduce_])

In [ ]:
if tuning_on==True:
  tuner.results_summary(num_trials=21)

In [ ]:
if tuning_on==True:
  print(tuner.get_best_hyperparameters(4)[0].values)
  print(tuner.get_best_hyperparameters(4)[1].values)
  print(tuner.get_best_hyperparameters(4)[2].values)
  print(tuner.get_best_hyperparameters(4)[3].values)
  print(tuner.get_best_hyperparameters(5)[4].values)
  print(tuner.get_best_hyperparameters(6)[5].values)
  print(tuner.get_best_hyperparameters(7)[6].values)

* {'main_activation': 'gelu', 'dropout': 0.375, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}
* {'main_activation': 'gelu', 'dropout': 0.15, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}
* {'main_activation': 'silu', 'dropout': 0.35, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}
* {'main_activation': 'relu', 'dropout': 0.375, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}
* {'main_activation': 'relu', 'dropout': 0.275, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}
* {'main_activation': 'relu', 'dropout': 0.375, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}
* {'main_activation': 'gelu', 'dropout': 0.30, 'reg': 0.0001, 'dense_layers': 128, 'conv_layers': 128}

* **Trial 34 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.375
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9520071744918823

* **Trial 30 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.15
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9539700746536255

* **Trial 22 summary**
Hyperparameters:
main_activation: silu
dropout: 0.35
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.954962134361267

* **Trial 24 summary**
Hyperparameters:
main_activation: relu
dropout: 0.375
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9550836086273193

* **Trial 11 summary**
Hyperparameters:
main_activation: relu
dropout: 0.275
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9551788568496704

* **Trial 40 summary**
Hyperparameters:
main_activation: relu
dropout: 0.375
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.955820918083191

* **Trial 37 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.30000000000000004
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9563628435134888

* **Trial 26 summary**
Hyperparameters:
main_activation: silu
dropout: 0.375
reg: 0.0001
dense_layers: 512
conv_layers: 128
Score: 1.956717610359192

* **Trial 15 summary**
Hyperparameters:
main_activation: silu
dropout: 0.275
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9571396112442017

* **Trial 25 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.325
reg: 0.0001
dense_layers: 256
conv_layers: 128
Score: 1.9579848051071167

* **Trial 44 summary**
Hyperparameters:
main_activation: relu
dropout: 0.375
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.958244800567627

* **Trial 19 summary**
Hyperparameters:
main_activation: relu
dropout: 0.30000000000000004
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9587109088897705

* **Trial 48 summary**
Hyperparameters:
main_activation: relu
dropout: 0.375
reg: 0.0001
dense_layers: 128
conv_layers: 128
Score: 1.9599860906600952

* **Trial 23 summary**
Hyperparameters:
main_activation: relu
dropout: 0.375
reg: 0.0001
dense_layers: 256
conv_layers: 128
Score: 1.9600061178207397

#### Fit The Model:

In [ ]:
#best_params = tuner.get_best_hyperparameters(1)[0].values
best_params = {'activation': 'silu', 'dropout': 0.35, 'reg': 0.0001, 'dense_layers': [128,64,64,64,32], 'conv_layers': [128,64,32,16]}

In [ ]:
test_results_df = run_experiment(X_train, X_test, create_model_v2, best_params, split=val_sets,
                                 experiment_name="conv_v2_nn", rs=42, target="bg+1:00",
                                 batch_size=256, num_epochs=200, learning_rate=0.00025)

In [ ]:
test_results_df.clip(2.7,22.5, inplace=True)

In [ ]:
plt.scatter(test_results_df[0],test_results_df[4])

In [ ]:
test_results_df["average"] = test_results_df.mean(axis=1)
test_results_df.max(axis=0),test_results_df.min(axis=0)

#### **STORE RESULTS**

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")

In [ ]:
for i, col in enumerate(test_results_df.columns):
  sub["bg+1:00"] = test_results_df[col].values
  sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v3_{col}_clipped.csv", index=False)
  print(sub.isna().sum())

## **2.4 MODEL 04**

### 3.1.1 Create Dataloader:

In [ ]:
X_train[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4",
          "enc_05_v4","enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7",
          "enc_08_v6","enc_04_v6","bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].describe()

In [ ]:
X_test[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4",
          "enc_05_v4","enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7",
          "enc_08_v6","enc_04_v6","bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].describe()

In [ ]:
gc.collect()

In [ ]:
def dataframe_to_dataset(dataframe, shuffle=False, batch_size=32):
    dataframe = dataframe.copy()
    target = dataframe["bg+1:00"]
    dataframe = dataframe.drop(columns=["bg+1:00"])

    static_df = dataframe.copy()


    ds = tf.data.Dataset.from_tensor_slices(((static_df["hour"].values,  # First input
                                              static_df["minute"].values,  # Second input
                                              static_df["cluster"].values,  # Third input
                                              static_df["cluster_pca"].values,  # Fourth input
                                              static_df[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4","enc_05_v4",
                                                        "enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7","enc_08_v6","enc_04_v6",
                                                         "bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].values),
                                              target))

    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [ ]:
train_ds = dataframe_to_dataset(X_train,batch_size=1)

In [ ]:
#X_train.filter(regex='insulin')

* **TEST THE DATALOADER:**

In [ ]:
for (x0, x1, x2, x3, x4), y in train_ds.take(1):
    print(x0.shape,x1.shape,x2.shape,x3.shape,x4.shape,y.shape)

In [ ]:
#X_train.iloc[:2000,:].to_csv("train_sample.csv")

### **2.1.2 Encoding**

In [ ]:
static_col = list(X_train.drop(columns=ts_fields+["bg+1:00"]).columns)

static_entries = {}

for col in static_col:
    static_entries[col] = X_train[col].nunique()

embed_dim = {'cluster_pca': 3, 'hour': 8, 'minute': 4, 'cluster': 8}
#static_entries

In [ ]:
def encode_tabular(inputs, list_categorical_nn, Cat_Feat_Entries=static_entries, num_dense_exp=False, embedding_dims=embed_dim, name="enc"):
    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for counter, feature_name in enumerate(inputs):

      vocabulary = Cat_Feat_Entries[feature_name]
      emb_dim = embedding_dims[feature_name]

      embedding = layers.Embedding(input_dim=vocabulary, output_dim=emb_dim, name=f"embedder_{counter}")
      # Convert the index values to embedding representations.
      encoded_categorical_feature = embedding(inputs[feature_name])

      encoded_categorical_feature_list.append(encoded_categorical_feature)

    return encoded_categorical_feature_list, numerical_feature_list

In [ ]:
X_train.info()

### **2.1.3 Create Models:**

In [ ]:
def create_model_v3(dropout=0.3,
                    activation="selu",
                    reg=0.0,
                    lr=0.001,
                    base=32,
                    rep=4,
                    summary=False):

  # --- Tabular Data Processing ---
  # Input layer for tabular data as a dictionary
  hour_input = keras.Input(shape=(1,), name="hour_input")
  minute_input = keras.Input(shape=(1,), name="minute_input")
  cluster_input = keras.Input(shape=(1,), name="cluster_input")
  pca_input = keras.Input(shape=(1,), name="cluster_pca")
  cont_input = keras.Input(shape=(19,), name="cont_inputs")

  # Lookup Layer for the p_num:
  # pca_lookup = layers.IntegerLookup(vocabulary=list(range(0,4)),mask_token=None,num_oov_indices=0,
  #                                  output_mode="int", name="lookup_pca")
  # pca_encoded = pca_lookup(pca_input)

  # Lookup Layer for the minutes:
  minute_lookup = layers.IntegerLookup(vocabulary=list(range(0,60,5)),mask_token=None,num_oov_indices=0,
                                       output_mode="int", name="lookup_min")

  minute_encoded = minute_lookup(minute_input)

  # Embedding layers for hour and minute
  hour_embedding = layers.Embedding(input_dim=24, output_dim=8, name="embed_hour")(hour_input)
  minute_embedding = layers.Embedding(input_dim=12, output_dim=6, name="embed_minute")(minute_encoded)
  pca_encoded_embedding = layers.Embedding(input_dim=4,output_dim=3, name="embed_pca")(pca_input)

  # Embedding layer for cluster
  cluster_embedding = layers.Embedding(input_dim=22, output_dim=8, name="embed_cluster")(cluster_input)

  # Flatten the embeddings
  hour_flat = layers.Flatten()(hour_embedding)
  minute_flat = layers.Flatten()(minute_embedding)
  cluster_flat = layers.Flatten()(cluster_embedding)
  pca_flat = layers.Flatten()(pca_encoded_embedding)

  # Concatenate all tabular features
  tabular_output = layers.concatenate([hour_flat, minute_flat, cluster_flat, pca_flat,cont_input],
                                      name="tabular_concat")
  all_feat = tabular_output

  # Dense layers for tabular data
  for num, den in enumerate(range(1,rep+1)):
    step = residual_block_dense(all_feat, base*den, activation=activation, normalization_type='batch', dropout_rate=dropout, name=f"block_{num}",  reg=reg,)
    all_feat = tf.keras.layers.concatenate([all_feat,step], name=f"concat_step_{num}")

  # --- Output Layer ---
  # Final dense layer for prediction
  output = layers.Dense(1, name="output")(all_feat)

  # --- Create and Compile the Model ---
  # Create the model
  model = keras.Model(inputs=[hour_input, minute_input, cluster_input, pca_input,cont_input], outputs=output)

  optimizer= keras.optimizers.Adam(learning_rate=lr)
  metric = RootMeanSquaredError(name="rmse", dtype=None)
  metric_mae = MeanAbsoluteError(name="mae", dtype=None)

  # Compile the model
  model.compile(optimizer=optimizer, loss="mse", metrics=[metric,metric_mae])

  # Print model summary
  if summary==True:
    model.summary()

  return model


#vocabulary = X_train["p_num"].unique().tolist()
model = create_model_v3(summary=True)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True,
           #rankdir="LR",
           expand_nested=True, show_layer_activations=True)

### 2.1.4 Training Functions:

In [ ]:
list(range(len(val_sets)))

#### Main Function:

In [ ]:
def run_experiment(X_train, X_test, model_constructor, best_params, split=val_sets, experiment_name="conv_v3_nn", rs=42, target="bg+1:00",
                   batch_size=64, num_epochs=200, learning_rate=0.001, target_scaler=target_scaler):

  test_predictions = np.zeros((len(X_test),1))
  test_results_df = pd.DataFrame(index=X_test.index, columns=list(range(len(val_sets))))

  all_mse = []
  all_rmse = []

  for i, val_index in enumerate(split):

    print(f"\nRunning CV {i}\n")
    ########################################################################## Prepare the Dataset:
    X_trn = X_train.drop(index=val_index)
    X_val = X_train.loc[val_index,:]

#    vocabulary = X_trn["p_num"].unique().tolist()

    X = X_trn.drop(columns=[target]).copy()
    y = X_trn[target].copy()

    val_X = X_val.drop(columns=[target]).copy()
    val_y = X_val[target].copy()

    X_test = X_test.copy()
    #################################################################### Prepare Datasets loaders:

    train_dataset = dataframe_to_dataset(X_trn, batch_size=batch_size, shuffle=True)
    valid_dataset = dataframe_to_dataset(X_val, batch_size=batch_size, shuffle=False)
    test_dataset = dataframe_to_dataset(X_test, batch_size=batch_size, shuffle=False)

    ##################################################################### Relevant Folders
    folders_experiment = f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/Glucose/{experiment_name}_{i}/"
    ##################################################################### Generate and Fit Model
    # Callbacks:
    checkpoint_filepath = folders_experiment + f'checkpoint/{experiment_name}.weights.h5'

    # Generate the Model:
    model = model_constructor(lr = learning_rate,
                              **best_params)

    print("Start training the model...")
    history = model.fit(train_dataset,
                        epochs=num_epochs,
                        callbacks=[keras.callbacks.EarlyStopping(monitor='val_rmse', patience=17, mode="min",
                                                  start_from_epoch=5,restore_best_weights=True),
                                   keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                    save_weights_only=True,
                                                    monitor="val_rmse",
                                                    mode='min',
                                                    save_best_only=True),
                                   keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5,
                                                          patience=5, min_lr=0.0001, mode="min")],
                        validation_data=valid_dataset)
    print("Model training finished")

    model.load_weights(checkpoint_filepath)
    model.evaluate(valid_dataset, verbose=0)

    plot_training_session(history)

    oof_res = model.predict(valid_dataset)
    test_pred = model.predict(test_dataset)

    print(f"Out-of-Fold Shapes: {val_y.shape},{oof_res.shape}")

    rmse_score = root_mean_squared_error(val_y, oof_res)

    val_pred = target_scaler.inverse_transform(oof_res)
    real_val_y = target_scaler.inverse_transform(val_y.values.reshape(-1,1))
    test_pred_real = target_scaler.inverse_transform(test_pred)

    rmse_score_original = root_mean_squared_error(real_val_y, val_pred)

    fig, axs = plt.subplots(1,1, figsize=(10,4))
    axs.scatter(val_pred, real_val_y)
    axs.set_title(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")
    plt.tight_layout()
    plt.show()

    print(f"Out-of-Fold RMSE Score Rebased: {round(rmse_score, 3)}%")
    print(f"Out-of-Fold RMSE Score Original: {round(rmse_score_original, 3)}%")

    ##################################################################### Save the Model
    model.save(f"{folders_experiment}/model_{experiment_name}.keras")

    ##################################################################### Create Model Output
    test_results_df.loc[:,i] = test_pred_real
    all_rmse.append(round(rmse_score_original, 3))

    gc.collect()

  ##################################################################### Create Model Output
  print(f"All Valuation RMSE: {all_rmse}")

  return test_results_df

#### Keras Tuner:

In [ ]:
tuning_on=True
gc.collect()

 Select a Validation set:

In [ ]:
valid_index = val_sets[0]
Xt = X_train.drop(index=valid_index)
Xv = X_train.loc[valid_index,:]

print(f"Train Shape: {Xt.shape}, Val Shape: {Xv.shape}")

In [ ]:
if tuning_on==True:

  train_dataset = dataframe_to_dataset(Xt, batch_size=256, shuffle=True)
  valid_dataset = dataframe_to_dataset(Xv, batch_size=256, shuffle=False)

In [ ]:
512/4

In [ ]:
if tuning_on==True:
  # Define the hyperparameter search space: EXPERIMENT 1
  hp = kt.HyperParameters()
  hp.Choice('activation', ["relu","silu","selu","gelu"]) #
  hp.Float('dropout',0.15,0.40, step=0.025) #
  hp.Float('reg',0.0001, 1.0,step=10,sampling="log") #
  hp.Choice('rep', values=[3,4,5,6])
#  hp.Choice('base', values=[32,64,128])

In [ ]:
def create_turner_model(hp):

  model = create_model_v3(dropout=hp.get('dropout'),
                          rep=hp.get('rep'),
                          activation=hp.get('activation'),
                          reg=hp.get('reg'),
                          lr=0.0025,
                          summary=False)
  return model

In [ ]:
if tuning_on==True:
  # Create a tuner and search for the best hyperparameters
  tuner = BayesianOptimization(create_turner_model,
                              objective=kt.Objective("val_rmse", "min"),
                              hyperparameters=hp, max_trials=50, overwrite=True)

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=7, mode="min", start_from_epoch=5)
  reduce_ = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, min_lr=0.0001, mode="min")

  tuner.search(train_dataset, validation_data=valid_dataset, epochs=31, callbacks=[stop_early,reduce_])

In [ ]:
if tuning_on==True:
  tuner.results_summary(num_trials=21)

In [ ]:
if tuning_on==True:
  print(tuner.get_best_hyperparameters(4)[0].values)
  print(tuner.get_best_hyperparameters(4)[1].values)
  print(tuner.get_best_hyperparameters(4)[2].values)
  print(tuner.get_best_hyperparameters(4)[3].values)
  print(tuner.get_best_hyperparameters(5)[4].values)
  print(tuner.get_best_hyperparameters(6)[5].values)
  print(tuner.get_best_hyperparameters(7)[6].values)

* {'activation': 'silu', 'dropout': 0.375, 'reg': 0.0001, 'rep': 4}
* {'activation': 'gelu', 'dropout': 0.35, 'reg': 0.001, 'rep': 4}
* {'activation': 'gelu', 'dropout': 0.275, 'reg': 0.0001, 'rep': 4}
* {'activation': 'gelu', 'dropout': 0.275, 'reg': 0.001, 'rep': 4}
* {'activation': 'silu', 'dropout': 0.375, 'reg': 0.001, 'rep': 4}
* {'activation': 'silu', 'dropout': 0.15, 'reg': 0.0001, 'rep': 3}
* {'activation': 'silu', 'dropout': 0.375, 'reg': 0.0001, 'rep': 4}


* **Trial 21 summary**
Hyperparameters:
main_activation: silu
dropout: 0.375
reg: 0.0001
dense_layers: 512
conv_layers: 64
Score: 1.9809489250183105

* **Trial 26 summary**
Hyperparameters:
main_activation: silu
dropout: 0.375
reg: 0.0001
dense_layers: 128
conv_layers: 64
Score: 1.9827678203582764

* **Trial 10 summary**
Hyperparameters:
main_activation: silu
dropout: 0.30000000000000004
reg: 0.001
dense_layers: 128
conv_layers: 64
Score: 1.9926979541778564

* **Trial 15 summary**
Hyperparameters:
main_activation: silu
dropout: 0.375
reg: 0.001
dense_layers: 256
conv_layers: 64
Score: 1.9969719648361206

* **Trial 18 summary**
Hyperparameters:
main_activation: relu
dropout: 0.2
reg: 0.0001
dense_layers: 64
conv_layers: 64
Score: 2.0100858211517334

#### Fit The Model:

In [ ]:
#best_params = tuner.get_best_hyperparameters(1)[0].values
best_params = {'activation': 'silu', 'dropout': 0.375, 'reg': 0.0001, 'rep': 4}

In [ ]:
test_results_df = run_experiment(X_train, X_test, create_model_v3, best_params, split=val_sets,
                                 experiment_name="conv_v3_nn", rs=42, target="bg+1:00",
                                 batch_size=256, num_epochs=200, learning_rate=0.00025)

In [ ]:
test_results_df.clip(2.7, 22.5, inplace=True)
test_results_df["average"] = test_results_df.mean(axis=1)
test_results_df

In [ ]:
check_res = test_results_df.clip(2.7,22.5)

In [ ]:
plt.scatter(check_res[0],check_res[2])

#### **STORE RESULTS**

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")

In [ ]:
for i, col in enumerate(test_results_df.columns):
  sub["bg+1:00"] = test_results_df[col].values
  sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v4_{col}_clipped.csv", index=False)
  print(sub.isna().sum())

## **2.5 MODEL 05**

### 3.1.1 Create Dataloader:

In [ ]:
gc.collect()

In [ ]:
# X_train[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4","enc_05_v4",
#          "enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7","enc_08_v6","enc_04_v6",
#          "bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].corr().style.background_gradient(cmap='coolwarm')

In [ ]:
def dataframe_to_dataset(dataframe, shuffle=False, batch_size=32):
    dataframe = dataframe.copy()
    target = dataframe["bg+1:00"]
    dataframe = dataframe.drop(columns=["bg+1:00"])

    static_df = dataframe.copy()


    ds = tf.data.Dataset.from_tensor_slices(((static_df["hour"].values,  # First input
                                              static_df["minute"].values,  # Second input
                                              static_df["cluster"].values,  # Third input
                                              static_df["cluster_pca"].values,  # Fourth input
                                              static_df[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4","enc_05_v4",
                                                        "enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7","enc_08_v6","enc_04_v6",
                                                         "bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].values),
                                              target))

    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [ ]:
train_ds = dataframe_to_dataset(X_train,batch_size=1)

* **TEST THE DATALOADER:**

In [ ]:
for (x0, x1, x2, x3, x4), y in train_ds.take(1):
    print(x0.shape,x1.shape,x2.shape,x3.shape,x4.shape,y.shape)

In [ ]:
#X_train.iloc[:2000,:].to_csv("train_sample.csv")

### **2.1.2 Encoding**

In [ ]:
static_col = list(X_train.drop(columns=ts_fields+["bg+1:00"]).columns)

static_entries = {}

for col in static_col:
    static_entries[col] = X_train[col].nunique()

embed_dim = {'cluster_pca': 3, 'hour': 8, 'minute': 4, 'cluster': 8}
#static_entries

In [ ]:
def encode_tabular(inputs, list_categorical_nn, Cat_Feat_Entries=static_entries, num_dense_exp=False, embedding_dims=embed_dim, name="enc"):
    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for counter, feature_name in enumerate(inputs):

      vocabulary = Cat_Feat_Entries[feature_name]
      emb_dim = embedding_dims[feature_name]

      embedding = layers.Embedding(input_dim=vocabulary, output_dim=emb_dim, name=f"embedder_{counter}")
      # Convert the index values to embedding representations.
      encoded_categorical_feature = embedding(inputs[feature_name])

      encoded_categorical_feature_list.append(encoded_categorical_feature)

    return encoded_categorical_feature_list, numerical_feature_list

In [ ]:
X_train.info()

### **2.1.3 Create Models:**

In [ ]:
def create_model_v4(dropout=0.3,
                    activation="selu",
                    reg=0.0,
                    lr=0.001,
                    dense_layers=[256,128,128,64],
                    norm="batch",
                    summary=False):

  # --- Tabular Data Processing ---
  # Input layer for tabular data as a dictionary
  hour_input = keras.Input(shape=(1,), name="hour_input")
  minute_input = keras.Input(shape=(1,), name="minute_input")
  cluster_input = keras.Input(shape=(1,), name="cluster_input")
  pca_input = keras.Input(shape=(1,), name="cluster_pca")
  cont_input = keras.Input(shape=(19,), name="cont_inputs")

  # Lookup Layer for the p_num:
  # pca_lookup = layers.IntegerLookup(vocabulary=list(range(0,4)),mask_token=None,num_oov_indices=0,
  #                                  output_mode="int", name="lookup_pca")
  # pca_encoded = pca_lookup(pca_input)

  # Lookup Layer for the minutes:
  minute_lookup = layers.IntegerLookup(vocabulary=list(range(0,60,5)),mask_token=None,num_oov_indices=0,
                                       output_mode="int", name="lookup_min")

  minute_encoded = minute_lookup(minute_input)

  # Embedding layers for hour and minute
  hour_embedding = layers.Embedding(input_dim=24, output_dim=8, name="embed_hour")(hour_input)
  minute_embedding = layers.Embedding(input_dim=12, output_dim=6, name="embed_minute")(minute_encoded)
  pca_encoded_embedding = layers.Embedding(input_dim=4,output_dim=3, name="embed_pca")(pca_input)

  # Embedding layer for cluster
  cluster_embedding = layers.Embedding(input_dim=22, output_dim=8, name="embed_cluster")(cluster_input)

  # Flatten the embeddings
  hour_flat = layers.Flatten()(hour_embedding)
  minute_flat = layers.Flatten()(minute_embedding)
  cluster_flat = layers.Flatten()(cluster_embedding)
  pca_flat = layers.Flatten()(pca_encoded_embedding)

  # Concatenate all tabular features
  tabular_output = layers.concatenate([hour_flat, minute_flat, cluster_flat, pca_flat,cont_input],
                                      name="tabular_concat")
  x = tabular_output

  # Dense layers for tabular data
  for num, den in enumerate(dense_layers):
    x = dense_block(den, dropout=dropout, activation=activation, reg=reg, name=f"Dense_block_{num}", normalization_type=norm)(x)

  # --- Output Layer ---
  # Final dense layer for prediction
  output = layers.Dense(1, name="output")(x)

  # --- Create and Compile the Model ---
  # Create the model
  model = keras.Model(inputs=[hour_input, minute_input, cluster_input, pca_input,cont_input], outputs=output)

  optimizer= keras.optimizers.Adam(learning_rate=lr)
  metric = RootMeanSquaredError(name="rmse", dtype=None)
  metric_mae = MeanAbsoluteError(name="mae", dtype=None)

  # Compile the model
  model.compile(optimizer=optimizer, loss="mse", metrics=[metric,metric_mae])

  # Print model summary
  if summary==True:
    model.summary()

  return model


#vocabulary = X_train["p_num"].unique().tolist()
model = create_model_v4(summary=True)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True,
           #rankdir="LR",
           expand_nested=True, show_layer_activations=True)

### 2.1.4 Training Functions:

In [ ]:
list(range(len(val_sets)))

#### Main Function:

In [ ]:
def run_experiment(X_train, X_test, model_constructor, best_params, split=val_sets, experiment_name="conv_v4_nn", rs=42, target="bg+1:00",
                   batch_size=64, num_epochs=200, learning_rate=0.001, target_scaler=target_scaler):

  test_predictions = np.zeros((len(X_test),1))
  test_results_df = pd.DataFrame(index=X_test.index, columns=list(range(len(val_sets))))

  all_mse = []
  all_rmse = []

  for i, val_index in enumerate(split):

    print(f"\nRunning CV {i}\n")
    ########################################################################## Prepare the Dataset:
    X_trn = X_train.drop(index=val_index)
    X_val = X_train.loc[val_index,:]

#    vocabulary = X_trn["p_num"].unique().tolist()

    X = X_trn.drop(columns=[target]).copy()
    y = X_trn[target].copy()

    val_X = X_val.drop(columns=[target]).copy()
    val_y = X_val[target].copy()

    X_test = X_test.copy()
    #################################################################### Prepare Datasets loaders:

    train_dataset = dataframe_to_dataset(X_trn, batch_size=batch_size, shuffle=True)
    valid_dataset = dataframe_to_dataset(X_val, batch_size=batch_size, shuffle=False)
    test_dataset = dataframe_to_dataset(X_test, batch_size=batch_size, shuffle=False)

    ##################################################################### Relevant Folders
    folders_experiment = f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/Glucose/{experiment_name}_{i}/"
    ##################################################################### Generate and Fit Model
    # Callbacks:
    checkpoint_filepath = folders_experiment + f'checkpoint/{experiment_name}.weights.h5'

    # Generate the Model:
    model = model_constructor(lr = learning_rate,
                              **best_params)

    print("Start training the model...")
    history = model.fit(train_dataset,
                        epochs=num_epochs,
                        callbacks=[keras.callbacks.EarlyStopping(monitor='val_rmse', patience=17, mode="min",
                                                  start_from_epoch=5,restore_best_weights=True),
                                   keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                    save_weights_only=True,
                                                    monitor="val_rmse",
                                                    mode='min',
                                                    save_best_only=True),
                                   keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5,
                                                          patience=5, min_lr=0.0001, mode="min")],
                        validation_data=valid_dataset)
    print("Model training finished")

    model.load_weights(checkpoint_filepath)
    model.evaluate(valid_dataset, verbose=0)

    plot_training_session(history)

    oof_res = model.predict(valid_dataset)
    test_pred = model.predict(test_dataset)

    print(f"Out-of-Fold Shapes: {val_y.shape},{oof_res.shape}")

    rmse_score = root_mean_squared_error(val_y, oof_res)

    val_pred = target_scaler.inverse_transform(oof_res)
    real_val_y = target_scaler.inverse_transform(val_y.values.reshape(-1,1))
    test_pred_real = target_scaler.inverse_transform(test_pred)

    rmse_score_original = root_mean_squared_error(real_val_y, val_pred)

    fig, axs = plt.subplots(1,1, figsize=(10,4))
    axs.scatter(val_pred, real_val_y)
    axs.set_title(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")
    plt.tight_layout()
    plt.show()

    print(f"Out-of-Fold RMSE Score Rebased: {round(rmse_score, 3)}%")
    print(f"Out-of-Fold RMSE Score Original: {round(rmse_score_original, 3)}%")

    ##################################################################### Save the Model
    model.save(f"{folders_experiment}/model_{experiment_name}.keras")

    ##################################################################### Create Model Output
    test_results_df.loc[:,i] = test_pred_real
    all_rmse.append(round(rmse_score_original, 3))

    gc.collect()

  ##################################################################### Create Model Output
  print(f"All Valuation RMSE: {all_rmse}")

  return test_results_df

#### Keras Tuner:

In [ ]:
tuning_on=True
gc.collect()

 Select a Validation set:

In [ ]:
valid_index = val_sets[0]
Xt = X_train.drop(index=valid_index)
Xv = X_train.loc[valid_index,:]

print(f"Train Shape: {Xt.shape}, Val Shape: {Xv.shape}")

In [ ]:
if tuning_on==True:

  train_dataset = dataframe_to_dataset(Xt, batch_size=256, shuffle=True)
  valid_dataset = dataframe_to_dataset(Xv, batch_size=256, shuffle=False)

In [ ]:
512/4

In [ ]:
if tuning_on==True:
  # Define the hyperparameter search space: EXPERIMENT 1
  hp = kt.HyperParameters()
  hp.Choice('main_activation', ["relu","silu","selu","gelu"]) #
  hp.Float('dropout',0.15,0.40, step=0.025) #
  hp.Float('reg',0.0001, 1.0,step=10,sampling="log") #
  hp.Choice('dense_layers', values=[512,256,128])
  hp.Choice('norm', values=["batch","layer"])

In [ ]:
def create_turner_model(hp):

  model = create_model_v4(dropout=hp.get('dropout'),
                    activation=hp.get('main_activation'),
                    reg=hp.get('reg'),
                    lr=0.001,
                    dense_layers=[hp.get('dense_layers'),int(hp.get('dense_layers')/2),int(hp.get('dense_layers')/2),int(hp.get('dense_layers')/4)],
                    norm=hp.get('norm'))

  return model

In [ ]:
if tuning_on==True:
  # Create a tuner and search for the best hyperparameters
  tuner = BayesianOptimization(create_turner_model,
                              objective=kt.Objective("val_rmse", "min"),
                              hyperparameters=hp, max_trials=50, overwrite=True)

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=7, mode="min", start_from_epoch=5)
  reduce_ = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, min_lr=0.0001, mode="min")

  tuner.search(train_dataset, validation_data=valid_dataset, epochs=31, callbacks=[stop_early,reduce_])

In [ ]:
if tuning_on==True:
  tuner.results_summary(num_trials=21)

In [ ]:
if tuning_on==True:
  print(tuner.get_best_hyperparameters(4)[0].values)
  print(tuner.get_best_hyperparameters(4)[1].values)
  print(tuner.get_best_hyperparameters(4)[2].values)
  print(tuner.get_best_hyperparameters(4)[3].values)

* {'main_activation': 'gelu', 'dropout': 0.2, 'reg': 0.001, 'dense_layers': 128, 'norm': 'batch'}
* {'main_activation': 'gelu', 'dropout': 0.175, 'reg': 0.001, 'dense_layers': 128, 'norm': 'batch'}
* {'main_activation': 'silu', 'dropout': 0.35, 'reg': 0.001, 'dense_layers': 256, 'norm': 'batch'}
* {'main_activation': 'silu', 'dropout': 0.35, 'reg': 0.001, 'dense_layers': 512, 'norm': 'batch'}

* **Trial 49 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.2
reg: 0.001
dense_layers: 128
norm: batch
Score: 0.6541268825531006

* **Trial 23 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.175
reg: 0.001
dense_layers: 128
norm: batch
Score: 0.6549822688102722

* **Trial 05 summary**
Hyperparameters:
main_activation: silu
dropout: 0.35
reg: 0.001
dense_layers: 256
norm: batch
Score: 0.6553966403007507

* **Trial 25 summary**
Hyperparameters:
main_activation: silu
dropout: 0.35
reg: 0.001
dense_layers: 512
norm: batch
Score: 0.6563018560409546

* **Trial 12 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.225
reg: 0.001
dense_layers: 128
norm: batch
Score: 0.6568679213523865

* **Trial 48 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.325
reg: 0.0001
dense_layers: 128
norm: layer
Score: 0.6574869155883789

* **Trial 17 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.30000000000000004
reg: 0.1
dense_layers: 512
norm: batch
Score: 0.6574987769126892

* **Trial 26 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.2
reg: 0.01
dense_layers: 128
norm: batch
Score: 0.6576610207557678

#### Fit The Model:

In [ ]:
#best_params = tuner.get_best_hyperparameters(1)[0].values
best_params = {'activation': 'gelu', 'dropout': 0.2, 'reg': 0.001, 'dense_layers': [128,64,64,32],'norm': 'batch'}

In [ ]:
test_results_df = run_experiment(X_train, X_test, create_model_v4, best_params, split=val_sets,
                                 experiment_name="dnn_v0", rs=42, target="bg+1:00",
                                 batch_size=256, num_epochs=200, learning_rate=0.00025)

In [ ]:
test_results_df.clip(2.7, 22.5, inplace=True)
test_results_df["average"] = test_results_df.mean(axis=1)
test_results_df

#### **STORE RESULTS**

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")

In [ ]:
for i, col in enumerate(test_results_df.columns):
  sub["bg+1:00"] = test_results_df[col].values
  sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_dnn_v0_{col}_clipped.csv", index=False)
  print(sub.isna().sum())

In [ ]:
test_results_df.min(axis=0),test_results_df.max(axis=0)

In [ ]:
plt.scatter(test_results_df[0],test_results_df[1])

## **2.6 MODEL 06**

### 3.1.1 Create Dataloader:

In [ ]:
gc.collect()

In [ ]:
# X_train[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4","enc_05_v4",
#          "enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7","enc_08_v6","enc_04_v6",
#          "bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].corr().style.background_gradient(cmap='coolwarm')

In [ ]:
def dataframe_to_dataset(dataframe, shuffle=False, batch_size=32):
    dataframe = dataframe.copy()
    target = dataframe["bg+1:00"]
    dataframe = dataframe.drop(columns=["bg+1:00"])

    static_df = dataframe.copy()


    ds = tf.data.Dataset.from_tensor_slices(((static_df["hour"].values,  # First input
                                              static_df["minute"].values,  # Second input
                                              static_df["cluster"].values,  # Third input
                                              static_df["cluster_pca"].values,  # Fourth input
                                              static_df[["PC_1","PC_2","PC_3","enc_04_v7","enc_01_v1","enc_07_v7","enc_01_v4","enc_05_v4",
                                                        "enc_03_v6","enc_05_v7","enc_06_v6","enc_01_v6","enc_06_v7","enc_08_v6","enc_04_v6",
                                                         "bg-0:00","insulin_av-0:00","brake-0:00","intake-0:00"]].values),
                                              target))

    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds

In [ ]:
train_ds = dataframe_to_dataset(X_train,batch_size=1)

* **TEST THE DATALOADER:**

In [ ]:
for (x0, x1, x2, x3, x4), y in train_ds.take(1):
    print(x0.shape,x1.shape,x2.shape,x3.shape,x4.shape,y.shape)

In [ ]:
#X_train.iloc[:2000,:].to_csv("train_sample.csv")

### **2.1.2 Encoding**

In [ ]:
static_col = list(X_train.drop(columns=ts_fields+["bg+1:00"]).columns)

static_entries = {}

for col in static_col:
    static_entries[col] = X_train[col].nunique()

embed_dim = {'cluster_pca': 3, 'hour': 8, 'minute': 4, 'cluster': 8}
#static_entries

In [ ]:
def encode_tabular(inputs, list_categorical_nn, Cat_Feat_Entries=static_entries, num_dense_exp=False, embedding_dims=embed_dim, name="enc"):
    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for counter, feature_name in enumerate(inputs):

      vocabulary = Cat_Feat_Entries[feature_name]
      emb_dim = embedding_dims[feature_name]

      embedding = layers.Embedding(input_dim=vocabulary, output_dim=emb_dim, name=f"embedder_{counter}")
      # Convert the index values to embedding representations.
      encoded_categorical_feature = embedding(inputs[feature_name])

      encoded_categorical_feature_list.append(encoded_categorical_feature)

    return encoded_categorical_feature_list, numerical_feature_list

In [ ]:
X_train.info()

### **2.1.3 Create Models:**

In [ ]:
def create_model_v5(dropout=0.3,
                    activation="selu",
                    reg=0.0,
                    lr=0.001,
                    dense_layers=[256,128,128,64],
                    norm="batch",
                    summary=False):

  # --- Tabular Data Processing ---
  # Input layer for tabular data as a dictionary
  hour_input = keras.Input(shape=(1,), name="hour_input")
  minute_input = keras.Input(shape=(1,), name="minute_input")
  cluster_input = keras.Input(shape=(1,), name="cluster_input")
  pca_input = keras.Input(shape=(1,), name="cluster_pca")
  cont_input = keras.Input(shape=(19,), name="cont_inputs")

  # Lookup Layer for the p_num:
  # pca_lookup = layers.IntegerLookup(vocabulary=list(range(0,4)),mask_token=None,num_oov_indices=0,
  #                                  output_mode="int", name="lookup_pca")
  # pca_encoded = pca_lookup(pca_input)

  # Lookup Layer for the minutes:
  minute_lookup = layers.IntegerLookup(vocabulary=list(range(0,60,5)),mask_token=None,num_oov_indices=0,
                                       output_mode="int", name="lookup_min")

  minute_encoded = minute_lookup(minute_input)

  # Embedding layers for hour and minute
  hour_embedding = layers.Embedding(input_dim=24, output_dim=8, name="embed_hour")(hour_input)
  minute_embedding = layers.Embedding(input_dim=12, output_dim=6, name="embed_minute")(minute_encoded)
  pca_encoded_embedding = layers.Embedding(input_dim=4,output_dim=3, name="embed_pca")(pca_input)

  # Embedding layer for cluster
  cluster_embedding = layers.Embedding(input_dim=22, output_dim=8, name="embed_cluster")(cluster_input)

  # Flatten the embeddings
  hour_flat = layers.Flatten()(hour_embedding)
  minute_flat = layers.Flatten()(minute_embedding)
  cluster_flat = layers.Flatten()(cluster_embedding)
  pca_flat = layers.Flatten()(pca_encoded_embedding)

  # Concatenate all tabular features
  x0 = layers.concatenate([hour_flat, minute_flat, cluster_flat, pca_flat,cont_input],
                                      name="tabular_concat")
  cross = x0
  for _ in dense_layers:
      units = cross.shape[-1]
      x = layers.Dense(units)(cross)
      cross = x0 * x + cross
  cross = layers.BatchNormalization()(cross)


  deep = x0
  for num, units in enumerate(dense_layers):
      deep = dense_block(units, dropout=dropout, activation=activation, reg=reg, name=f"Dense_block_{num}", normalization_type=norm)(deep)

  merged = layers.concatenate([cross, deep])

  # --- Output Layer ---
  # Final dense layer for prediction
  output = layers.Dense(1, name="output")(merged)

  # --- Create and Compile the Model ---
  # Create the model
  model = keras.Model(inputs=[hour_input, minute_input, cluster_input, pca_input,cont_input], outputs=output)

  optimizer= keras.optimizers.Adam(learning_rate=lr)
  metric = RootMeanSquaredError(name="rmse", dtype=None)
  metric_mae = MeanAbsoluteError(name="mae", dtype=None)

  # Compile the model
  model.compile(optimizer=optimizer, loss="mse", metrics=[metric,metric_mae])

  # Print model summary
  if summary==True:
    model.summary()

  return model


#vocabulary = X_train["p_num"].unique().tolist()
model = create_model_v5(summary=True)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True,
           rankdir="LR",
           expand_nested=True, show_layer_activations=True)

### 2.1.4 Training Functions:

In [ ]:
list(range(len(val_sets)))

#### Main Function:

In [ ]:
def run_experiment(X_train, X_test, model_constructor, best_params, split=val_sets, experiment_name="conv_v5_nn", rs=42, target="bg+1:00",
                   batch_size=64, num_epochs=200, learning_rate=0.001, target_scaler=target_scaler):

  test_predictions = np.zeros((len(X_test),1))
  test_results_df = pd.DataFrame(index=X_test.index, columns=list(range(len(val_sets))))

  all_mse = []
  all_rmse = []

  for i, val_index in enumerate(split):

    print(f"\nRunning CV {i}\n")
    ########################################################################## Prepare the Dataset:
    X_trn = X_train.drop(index=val_index)
    X_val = X_train.loc[val_index,:]

#    vocabulary = X_trn["p_num"].unique().tolist()

    X = X_trn.drop(columns=[target]).copy()
    y = X_trn[target].copy()

    val_X = X_val.drop(columns=[target]).copy()
    val_y = X_val[target].copy()

    X_test = X_test.copy()
    #################################################################### Prepare Datasets loaders:

    train_dataset = dataframe_to_dataset(X_trn, batch_size=batch_size, shuffle=True)
    valid_dataset = dataframe_to_dataset(X_val, batch_size=batch_size, shuffle=False)
    test_dataset = dataframe_to_dataset(X_test, batch_size=batch_size, shuffle=False)

    ##################################################################### Relevant Folders
    folders_experiment = f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/Glucose/{experiment_name}_{i}/"
    ##################################################################### Generate and Fit Model
    # Callbacks:
    checkpoint_filepath = folders_experiment + f'checkpoint/{experiment_name}.weights.h5'

    # Generate the Model:
    model = model_constructor(lr = learning_rate,
                              **best_params)

    print("Start training the model...")
    history = model.fit(train_dataset,
                        epochs=num_epochs,
                        callbacks=[keras.callbacks.EarlyStopping(monitor='val_rmse', patience=17, mode="min",
                                                  start_from_epoch=5,restore_best_weights=True),
                                   keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                    save_weights_only=True,
                                                    monitor="val_rmse",
                                                    mode='min',
                                                    save_best_only=True),
                                   keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5,
                                                          patience=5, min_lr=0.0001, mode="min")],
                        validation_data=valid_dataset)
    print("Model training finished")

    model.load_weights(checkpoint_filepath)
    model.evaluate(valid_dataset, verbose=0)

    plot_training_session(history)

    oof_res = model.predict(valid_dataset)
    test_pred = model.predict(test_dataset)

    print(f"Out-of-Fold Shapes: {val_y.shape},{oof_res.shape}")

    rmse_score = root_mean_squared_error(val_y, oof_res)

    val_pred = target_scaler.inverse_transform(oof_res)
    real_val_y = target_scaler.inverse_transform(val_y.values.reshape(-1,1))
    test_pred_real = target_scaler.inverse_transform(test_pred)

    rmse_score_original = root_mean_squared_error(real_val_y, val_pred)

    fig, axs = plt.subplots(1,1, figsize=(10,4))
    axs.scatter(val_pred, real_val_y)
    axs.set_title(f"Out-of-Fold RMSE Score: {round(rmse_score, 3)}%")
    plt.tight_layout()
    plt.show()

    print(f"Out-of-Fold RMSE Score Rebased: {round(rmse_score, 3)}%")
    print(f"Out-of-Fold RMSE Score Original: {round(rmse_score_original, 3)}%")

    ##################################################################### Save the Model
    model.save(f"{folders_experiment}/model_{experiment_name}.keras")

    ##################################################################### Create Model Output
    test_results_df.loc[:,i] = test_pred_real
    all_rmse.append(round(rmse_score_original, 3))

    gc.collect()

  ##################################################################### Create Model Output
  print(f"All Valuation RMSE: {all_rmse}")

  return test_results_df

#### Keras Tuner:

In [ ]:
tuning_on=True
gc.collect()

 Select a Validation set:

In [ ]:
valid_index = val_sets[0]
Xt = X_train.drop(index=valid_index)
Xv = X_train.loc[valid_index,:]

print(f"Train Shape: {Xt.shape}, Val Shape: {Xv.shape}")

In [ ]:
if tuning_on==True:

  train_dataset = dataframe_to_dataset(Xt, batch_size=256, shuffle=True)
  valid_dataset = dataframe_to_dataset(Xv, batch_size=256, shuffle=False)

In [ ]:
if tuning_on==True:
  # Define the hyperparameter search space: EXPERIMENT 1
  hp = kt.HyperParameters()
  hp.Choice('activation', ["relu","silu","gelu","mish"]) #
  hp.Float('dropout',0.25,0.40, step=0.025) #
  hp.Float('reg',0.0001, 0.1,step=10,sampling="log") #
  hp.Choice('dense_layers', values=[1028,512,256])
  hp.Choice('norm', values=["batch"])

In [ ]:
def create_turner_model(hp):

  model = create_model_v5(dropout=hp.get('dropout'),
                    activation=hp.get('activation'),
                    reg=hp.get('reg'),
                    lr=0.0005,
                    dense_layers=[hp.get('dense_layers'),int(hp.get('dense_layers')/2),int(hp.get('dense_layers')/4)],
                    norm=hp.get('norm'))

  return model

In [ ]:
if tuning_on==True:
  # Create a tuner and search for the best hyperparameters
  tuner = BayesianOptimization(create_turner_model,
                              objective=kt.Objective("val_rmse", "min"),
                              hyperparameters=hp, max_trials=50, overwrite=True)

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=7, mode="min", start_from_epoch=5)
  reduce_ = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, min_lr=0.0001, mode="min")

  tuner.search(train_dataset, validation_data=valid_dataset, epochs=31, callbacks=[stop_early,reduce_])

    Trial 19 Complete [00h 02m 35s]
    val_rmse: 0.6537328958511353

    Best val_rmse So Far: 0.6537328958511353
    Total elapsed time: 00h 39m 28s

    Search: Running Trial #20

    Value             |Best Value So Far |Hyperparameter
    gelu              |relu              |activation
    0.225             |0.325             |dropout
    0.0001            |0.01              |reg
    512               |512               |dense_layers
    batch             |batch             |norm

In [ ]:
if tuning_on==True:
  tuner.results_summary(num_trials=21)

In [ ]:
if tuning_on==True:
  print(tuner.get_best_hyperparameters(4)[0].values)
  print(tuner.get_best_hyperparameters(4)[1].values)
  print(tuner.get_best_hyperparameters(4)[2].values)
  print(tuner.get_best_hyperparameters(4)[3].values)

* {'main_activation': 'gelu', 'dropout': 0.2, 'reg': 0.001, 'dense_layers': 128, 'norm': 'batch'}
* {'main_activation': 'gelu', 'dropout': 0.175, 'reg': 0.001, 'dense_layers': 128, 'norm': 'batch'}
* {'main_activation': 'silu', 'dropout': 0.35, 'reg': 0.001, 'dense_layers': 256, 'norm': 'batch'}
* {'main_activation': 'silu', 'dropout': 0.35, 'reg': 0.001, 'dense_layers': 512, 'norm': 'batch'}

* **Trial 49 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.2
reg: 0.001
dense_layers: 128
norm: batch
Score: 0.6541268825531006

* **Trial 23 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.175
reg: 0.001
dense_layers: 128
norm: batch
Score: 0.6549822688102722

* **Trial 05 summary**
Hyperparameters:
main_activation: silu
dropout: 0.35
reg: 0.001
dense_layers: 256
norm: batch
Score: 0.6553966403007507

* **Trial 25 summary**
Hyperparameters:
main_activation: silu
dropout: 0.35
reg: 0.001
dense_layers: 512
norm: batch
Score: 0.6563018560409546

* **Trial 12 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.225
reg: 0.001
dense_layers: 128
norm: batch
Score: 0.6568679213523865

* **Trial 48 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.325
reg: 0.0001
dense_layers: 128
norm: layer
Score: 0.6574869155883789

* **Trial 17 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.30000000000000004
reg: 0.1
dense_layers: 512
norm: batch
Score: 0.6574987769126892

* **Trial 26 summary**
Hyperparameters:
main_activation: gelu
dropout: 0.2
reg: 0.01
dense_layers: 128
norm: batch
Score: 0.6576610207557678

#### Fit The Model:

In [ ]:
#best_params = tuner.get_best_hyperparameters(1)[0].values
best_params = {'activation': 'silu', 'dropout': 0.375, 'reg': 0.01, 'dense_layers': [512,256,128], 'norm': 'batch'}

In [ ]:
test_results_df = run_experiment(X_train, X_test, create_model_v5, best_params, split=val_sets,
                                 experiment_name="dnn_v0", rs=42, target="bg+1:00",
                                 batch_size=256, num_epochs=200, learning_rate=0.00025)

In [ ]:
test_results_df.clip(2.7, 22.5, inplace=True)
test_results_df["average"] = test_results_df.mean(axis=1)
test_results_df

#### **STORE RESULTS**

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")

In [ ]:
for i, col in enumerate(test_results_df.columns):
  sub["bg+1:00"] = test_results_df[col].values
  sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_crossdeep_v0_{col}_clipped.csv", index=False)
  print(sub.isna().sum())

In [ ]:
test_results_df.min(axis=0),test_results_df.max(axis=0)

In [ ]:
plt.scatter(test_results_df[0],test_results_df[1])